<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/Chapter-3/Chapter4_Empirical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install
!pip install stargazer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import
from google.colab import auth
# from google.cloud import bigquery
# from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')

import os
path = '/content/drive/My Drive/4 Vaccination and Election Results/Data'
os.chdir(path)

# project = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
# location = 'US' # Location inserted based on the query results selected to explore
# client = bigquery.Client(project=project, location=location)
# data_table.enable_dataframe_formatter()
# auth.authenticate_user()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from stargazer.stargazer import Stargazer

# from zipfile import ZipFile

SEED = np.random.seed(42)

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

# auth.authenticate_user()
# project_id = "phd-dissertation-343113"
# !gcloud config set project {project_id}

Mounted at /content/drive


In [3]:
df = pd.read_csv('df.csv')
df.head(10)

,id_municipio,id_municipio_tse,Name,State,Region,Population,Dose1 Applied,Dose2 Applied,Dose3 Applied,Dose4 Applied,...,Catholic,Evangelic,Pentecostal Evangelic,Umbanda/Candomble,No religion,Education 8-10y,Education 11-14y,Education 15y plus,Income (BRL),Education 8y less
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,0.778335,0.655001,0.103260,0.000266,...,0.642388,0.114319,0.165616,0.000000,0.121069,10.13,6.52,0.74,1310.78,81.23
1,1100023,78,Ariquemes,RO,Norte,111148,0.660795,0.527999,0.110672,0.003176,...,0.329021,0.054720,0.179976,0.000000,0.163602,15.57,10.28,1.81,1849.30,71.52
2,1100031,450,Cabixi,RO,Norte,5067,0.881389,0.737320,0.114466,0.008881,...,0.761200,0.060193,0.288534,0.000000,0.113479,9.87,5.25,0.29,1232.03,82.53
3,1100049,94,Cacoal,RO,Norte,86416,0.845156,0.635554,0.107816,0.001365,...,0.388065,0.130994,0.184804,0.000000,0.117525,13.75,11.12,1.97,1853.64,71.00
4,1100056,272,Cerejeiras,RO,Norte,16088,0.781701,0.648993,0.145699,0.001740,...,0.627797,0.100945,0.180756,0.000000,0.090627,12.36,8.83,0.64,1573.62,76.52
5,1100064,230,Colorado do Oeste,RO,Norte,15213,0.859199,0.678170,0.112798,0.001775,...,0.704792,0.058437,0.254190,0.000000,0.098666,12.94,10.23,1.32,1518.89,73.01
6,1100072,655,Corumbiara,RO,Norte,7052,0.832388,0.724759,0.155559,0.000425,...,0.823880,0.057856,0.233976,0.000000,0.106069,5.95,2.95,0.34,1188.16,89.41
7,1100080,213,Costa Marques,RO,Norte,19255,0.461646,0.368476,0.060608,0.000675,...,0.448195,0.061023,0.123448,0.000208,0.034329,9.72,6.54,1.31,1225.56,81.35
8,1100098,256,Espigão D'Oeste,RO,Norte,33009,0.691690,0.530128,0.053379,0.000515,...,0.382805,0.208004,0.129540,0.000000,0.106304,10.91,6.00,0.73,1455.66,79.78
9,1100106,19,Guajará-Mirim,RO,Norte,46930,0.634626,0.439740,0.041168,0.000384,...,0.512849,0.093394,0.132325,0.000192,0.083273,17.19,12.16,2.32,1646.55,64.65


In [4]:
df['Outer wing 18'] = df['Far-left 18'] + df['Far-right18']
df['outer wing 22'] = df['Far-left 22'] + df['Far-right 22']

In [5]:
columns_corr = ['Doses Applied', 'Doses Received',  'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'PL coligation 22', 
       'Population', 'elderly(>70yo)',  'Rural pop.',   'Pop. density', 'White', 'Black', 'Brown',
       'Catholic', 'Evangelic', 'No religion', 'Education 8y less', 'Education 15y plus', 'Income (BRL)']

corr = df[columns_corr].corr(method='pearson')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

<ipython-input-5-f626901f56d4>:10: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,Doses Applied,Doses Received,Dose3 Applied,Dose3 Received,Vaccination coverage,Polio coverage,Bolsonaro Share 18,Bolsonaro Share 22,Far-left 18,Far-right 22,PL coligation 22,Population,elderly(>70yo),Pop. density,White,Brown,Catholic,Evangelic,No religion,Education 8y less,Income (BRL)
Doses Applied,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Doses Received,0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose3 Applied,0.78,0.75,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose3 Received,0.72,0.82,0.96,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Vaccination coverage,0.33,0.38,0.34,0.35,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Polio coverage,0.27,0.31,0.28,0.28,0.90,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Bolsonaro Share 18,0.39,0.33,0.36,0.38,0.23,0.14,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Bolsonaro Share 22,0.35,0.28,0.31,0.33,0.22,0.13,0.98,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Far-left 18,0.05,-0.01,-0.01,-0.01,-0.11,-0.11,0.24,0.20,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Far-right 22,0.28,0.25,0.21,0.22,0.21,0.16,0.51,0.44,0.24,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [6]:
print(corr.round(2).to_latex())

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrr}
\toprule
{} &  Doses Applied &  Doses Received &  Dose3 Applied &  Dose3 Received &  Vaccination coverage &  Polio coverage &  Bolsonaro Share 18 &  Bolsonaro Share 22 &  Far-left 18 &  Far-right 22 &  PL coligation 22 &  Population &  elderly(>70yo) &  Pop. density &  White &  Brown &  Catholic &  Evangelic &  No religion &  Education 8y less &  Income (BRL) \\
\midrule
Doses Applied        &            NaN &             NaN &            NaN &             NaN &                   NaN &             NaN &                 NaN &                 NaN &          NaN &           NaN &               NaN &         NaN &             NaN &           NaN &    NaN &    NaN &       NaN &        NaN &          NaN &                NaN &           NaN \\
Doses Received       &           0.78 &             NaN &            NaN &             NaN &                   NaN &             NaN &                 NaN &                 NaN &          NaN &           NaN &   

<ipython-input-6-aca0328c279e>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(corr.round(2).to_latex())


In [256]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Ciro Share 18', 'Far-left 22',
       'Far-right 22', 'PL coligation 22', 'Tebet Share 22',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage',
       'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous',
       'Pop. density', 'Catholic', 'Evangelic', 'Pentecostal Evangelic',
       'Umbanda/Candomble', 'No religion', 'Education 8-10y',
       'Education 11-14y', 'Education 15y plus', 'Income (BRL)',
       'Education 8y less', 'Outer wing 18', 'outer wing 22'],
      dtype='object')

In [324]:
y_vars = 'Doses Applied', 'Doses Received', 'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage'
x_vars1 = ('Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
# x_vars1 = ('PSL coligation 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
x_vars2 = ('Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
x_vars3 = ('outer wing 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

x_vars4 = ('Ciro Share 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

x_vars5 = ('Tebet Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

In [325]:
# y_vars = 'Doses Applied', 'Doses Received', 'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage'

# x_vars1 = ('Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

# x_vars2 = ('Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

# x_vars3 = ('outer wing 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

# x_vars4 = ('Ciro Share 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

# x_vars5 = ('Tebet Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

# x_vars6 = ('Bolsonaro Share 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

In [326]:
df[list(x_vars1)] = df[list(x_vars1)].replace('-', 0).astype('float64')

In [327]:
models = []

In [328]:
# setting DF for OLS
X1 = sm.add_constant(df[list(x_vars1)])
X2 = sm.add_constant(df[list(x_vars2)])
X3 = sm.add_constant(df[list(x_vars3)])
X4 = sm.add_constant(df[list(x_vars4)])
X5 = sm.add_constant(df[list(x_vars5)])
X6 = sm.add_constant(df[list(x_vars6)])

# target1 (y1) as Doses Applied
y1 = df[y_vars[0]].values.reshape(-1, 1)

# target2 (y2) as Doses Received
y2 = df[y_vars[1]].values.reshape(-1, 1)

# target3 (y3) as Dose3 Applied
y3 = df[y_vars[2]].values.reshape(-1, 1)

# target4 (y24 as Dose3 Received
y4 = df[y_vars[3]].values.reshape(-1, 1)

# target5 (y5) as General vaxx coverage
y5 = df[y_vars[4]].values.reshape(-1, 1)

# target6 (y6) as Polio vaxx coverage
y6 = df[y_vars[5]].values.reshape(-1, 1)



In [329]:
# Doses aplicadas ~ Bolsonaro Share '22

# Create the OLS model with a different name
model1 = sm.OLS(y1, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     345.8
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                -1227.7
No. Observations:                5507   AIC:                             2491.
Df Residuals:                    5489   BIC:                             2610.
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.6342      0.696     -2.347      0.019      -2.999      -0.269
Bolsonaro Share 22       -0.0983      0.049     -1.997      0.046      -0.195      -0.002
elderly(>70yo)            0.0471      0.004     13.143      0.000       0.040       0.054
Rural pop.             1.076e-05      0.000      0.038      0.970      -0.001       0.001
White                    -0.0227      0.005     -5.021      0.000      -0.032      -0.014
Black                    -0.0244      0.005     -5.057      0.000      -0.034      -0.015
Brown                    -0.0280      0.005     -6.122      0.000      -0.037      -0.019
Indigenous               -0.0263      0.005     -5.718      0.000      -0.035      -0.017
Pop. density          -7.828e-06   1.09e-05     -0.715      0.474   -2.93e-05    1.36e-05
Catholic                  1.0153      0.071     14.312      0.000       0.876       1.154
Evangelic                 0.5988      0.109      5.506      0.000       0.386       0.812
Pentecostal Evangelic     1.0157      0.104      9.787      0.000       0.812       1.219
No religion               1.1705      0.142      8.220      0.000       0.891       1.450
Education 8y less         0.0424      0.006      7.674      0.000       0.032       0.053
Education 8-10y           0.0485      0.006      8.358      0.000       0.037       0.060
Education 11-14y          0.0524      0.006      9.230      0.000       0.041       0.063
Education 15y plus        0.0147      0.007      2.055      0.040       0.001       0.029
Income (BRL)              0.0002   1.84e-05      8.627      0.000       0.000       0.000
==============================================================================
Omnibus:                     3539.661   Durbin-Watson:                   1.450
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           549989.772
Skew:                           2.085   Prob(JB):                         0.00
Kurtosis:                      51.780   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [330]:
# Doses recebidas ~ Bolsonaro Share '22

# Create the OLS model with a different name
model2 = sm.OLS(y2, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     335.6
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                -319.23
No. Observations:                5507   AIC:                             674.5
Df Residuals:                    5489   BIC:                             793.5
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.3094      0.498      4.640      0.000       1.334       3.285
Bolsonaro Share 22       -0.1237      0.041     -3.039      0.002      -0.204      -0.044
elderly(>70yo)            0.0647      0.004     14.759      0.000       0.056       0.073
Rural pop.               -0.0012      0.000     -4.570      0.000      -0.002      -0.001
White                    -0.0204      0.004     -5.070      0.000      -0.028      -0.013
Black                    -0.0259      0.004     -6.008      0.000      -0.034      -0.017
Brown                    -0.0249      0.004     -6.110      0.000      -0.033      -0.017
Indigenous               -0.0236      0.004     -5.537      0.000      -0.032      -0.015
Pop. density          -6.999e-06   6.95e-06     -1.007      0.314   -2.06e-05    6.62e-06
Catholic                  1.1891      0.119      9.965      0.000       0.955       1.423
Evangelic                 0.6574      0.148      4.434      0.000       0.367       0.948
Pentecostal Evangelic     1.0957      0.185      5.907      0.000       0.732       1.459
No religion               1.1230      0.169      6.664      0.000       0.793       1.453
Education 8y less        -0.0009      0.003     -0.273      0.785      -0.007       0.006
Education 8-10y           0.0096      0.004      2.546      0.011       0.002       0.017
Education 11-14y          0.0071      0.004      1.836      0.066      -0.000       0.015
Education 15y plus       -0.0289      0.005     -5.361      0.000      -0.039      -0.018
Income (BRL)              0.0001   1.81e-05      7.384      0.000    9.84e-05       0.000
==============================================================================
Omnibus:                      764.594   Durbin-Watson:                   1.825
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3799.834
Skew:                           0.577   Prob(JB):                         0.00
Kurtosis:                       6.902   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [331]:
# 3ª dose aplicada ~ Bolsonaro Share '22

# Create the OLS model with a different name
model3 = sm.OLS(y3, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     262.4
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                 7531.8
No. Observations:                5507   AIC:                        -1.503e+04
Df Residuals:                    5489   BIC:                        -1.491e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3881      0.120     -3.244      0.001      -0.623      -0.154
Bolsonaro Share 22       -0.0284      0.010     -2.860      0.004      -0.048      -0.009
elderly(>70yo)            0.0176      0.001     21.588      0.000       0.016       0.019
Rural pop.                0.0002   6.16e-05      3.785      0.000       0.000       0.000
White                    -0.0009      0.001     -0.963      0.336      -0.003       0.001
Black                    -0.0023      0.001     -2.248      0.025      -0.004      -0.000
Brown                    -0.0019      0.001     -1.921      0.055      -0.004    3.86e-05
Indigenous               -0.0010      0.001     -1.054      0.292      -0.003       0.001
Pop. density          -1.802e-06   1.23e-06     -1.468      0.142   -4.21e-06    6.05e-07
Catholic                  0.0861      0.014      6.285      0.000       0.059       0.113
Evangelic                 0.0067      0.028      0.240      0.810      -0.048       0.062
Pentecostal Evangelic     0.1220      0.029      4.141      0.000       0.064       0.180
No religion               0.1120      0.027      4.195      0.000       0.060       0.164
Education 8y less         0.0044      0.001      5.107      0.000       0.003       0.006
Education 8-10y           0.0058      0.001      5.769      0.000       0.004       0.008
Education 11-14y          0.0062      0.001      6.615      0.000       0.004       0.008
Education 15y plus       -0.0007      0.001     -0.481      0.631      -0.003       0.002
Income (BRL)           2.715e-05   4.88e-06      5.563      0.000    1.76e-05    3.67e-05
==============================================================================
Omnibus:                     2078.456   Durbin-Watson:                   1.789
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23356.579
Skew:                           1.480   Prob(JB):                         0.00
Kurtosis:                      12.645   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [332]:
# 3ª dose recebida ~ Bolsonaro Share '22

# Create the OLS model with a different name
model4 = sm.OLS(y4, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     304.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                 7999.1
No. Observations:                5507   AIC:                        -1.596e+04
Df Residuals:                    5489   BIC:                        -1.584e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.2364      0.105     -2.256      0.024      -0.442      -0.031
Bolsonaro Share 22       -0.0200      0.008     -2.373      0.018      -0.036      -0.003
elderly(>70yo)            0.0177      0.001     23.125      0.000       0.016       0.019
Rural pop.                0.0001   5.51e-05      2.326      0.020    2.02e-05       0.000
White                    -0.0010      0.001     -1.087      0.277      -0.003       0.001
Black                    -0.0024      0.001     -2.555      0.011      -0.004      -0.001
Brown                    -0.0019      0.001     -2.125      0.034      -0.004      -0.000
Indigenous               -0.0010      0.001     -1.104      0.270      -0.003       0.001
Pop. density          -1.177e-06   1.13e-06     -1.041      0.298   -3.39e-06    1.04e-06
Catholic                  0.0996      0.015      6.803      0.000       0.071       0.128
Evangelic                 0.0145      0.027      0.543      0.587      -0.038       0.067
Pentecostal Evangelic     0.1229      0.030      4.044      0.000       0.063       0.183
No religion               0.1129      0.025      4.433      0.000       0.063       0.163
Education 8y less         0.0028      0.001      3.983      0.000       0.001       0.004
Education 8-10y           0.0043      0.001      5.303      0.000       0.003       0.006
Education 11-14y          0.0047      0.001      5.730      0.000       0.003       0.006
Education 15y plus       -0.0022      0.001     -1.816      0.069      -0.005       0.000
Income (BRL)           2.496e-05   4.21e-06      5.924      0.000    1.67e-05    3.32e-05
==============================================================================
Omnibus:                     1513.225   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9829.082
Skew:                           1.151   Prob(JB):                         0.00
Kurtosis:                       9.126   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [333]:
# Cobertura vacinal ~ Bolsonaro Share '22

# Create the OLS model with a different name
model5 = sm.OLS(y5, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.191
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     63.34
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.76e-198
Time:                        13:55:15   Log-Likelihood:                -24774.
No. Observations:                5507   AIC:                         4.958e+04
Df Residuals:                    5489   BIC:                         4.970e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   151.8909     41.330      3.675      0.000      70.885     232.897
Bolsonaro Share 22       16.2145      3.469      4.674      0.000       9.416      23.013
elderly(>70yo)            2.4174      0.264      9.164      0.000       1.900       2.935
Rural pop.               -0.0162      0.021     -0.780      0.435      -0.057       0.024
White                    -1.1944      0.337     -3.541      0.000      -1.855      -0.533
Black                    -1.3412      0.357     -3.752      0.000      -2.042      -0.641
Brown                    -1.3258      0.340     -3.895      0.000      -1.993      -0.659
Indigenous               -1.4714      0.343     -4.289      0.000      -2.144      -0.799
Pop. density             -0.0014      0.001     -2.218      0.027      -0.003      -0.000
Catholic                 11.2929      3.141      3.595      0.000       5.137      17.449
Evangelic                -4.5235      8.022     -0.564      0.573     -20.247      11.200
Pentecostal Evangelic     7.0298      6.436      1.092      0.275      -5.585      19.644
No religion             -55.5050      9.692     -5.727      0.000     -74.501     -36.509
Education 8y less         0.2022      0.280      0.722      0.470      -0.346       0.751
Education 8-10y           0.2240      0.315      0.711      0.477      -0.394       0.842
Education 11-14y         -0.3664      0.312     -1.176      0.240      -0.977       0.244
Education 15y plus       -2.7009      0.474     -5.694      0.000      -3.631      -1.771
Income (BRL)              0.0100      0.002      6.375      0.000       0.007       0.013
==============================================================================
Omnibus:                      545.195   Durbin-Watson:                   1.890
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2130.295
Skew:                           0.439   Prob(JB):                         0.00
Kurtosis:                       5.917   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [334]:
# Cobertura contra Polio ~ Bolsonaro share '22

# Create the OLS model with a different name
model6 = sm.OLS(y6, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     36.64
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.29e-114
Time:                        13:55:15   Log-Likelihood:                -26168.
No. Observations:                5507   AIC:                         5.237e+04
Df Residuals:                    5489   BIC:                         5.249e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   190.2416     53.268      3.571      0.000      85.839     294.644
Bolsonaro Share 22       12.4666      4.706      2.649      0.008       3.243      21.690
elderly(>70yo)            2.8601      0.361      7.926      0.000       2.153       3.567
Rural pop.               -0.0145      0.026     -0.563      0.574      -0.065       0.036
White                    -1.2818      0.436     -2.937      0.003      -2.137      -0.426
Black                    -1.4982      0.460     -3.254      0.001      -2.401      -0.596
Brown                    -1.4417      0.439     -3.283      0.001      -2.302      -0.581
Indigenous               -1.7050      0.440     -3.876      0.000      -2.567      -0.843
Pop. density             -0.0010      0.001     -1.376      0.169      -0.002       0.000
Catholic                 11.6948      3.946      2.964      0.003       3.961      19.429
Evangelic               -13.1078     13.051     -1.004      0.315     -38.688      12.472
Pentecostal Evangelic     5.1778      8.460      0.612      0.541     -11.403      21.759
No religion             -44.0390     12.508     -3.521      0.000     -68.553     -19.525
Education 8y less         0.0649      0.365      0.178      0.859      -0.650       0.780
Education 8-10y           0.0127      0.418      0.030      0.976      -0.806       0.831
Education 11-14y         -0.5517      0.395     -1.396      0.163      -1.326       0.223
Education 15y plus       -2.8115      0.621     -4.531      0.000      -4.028      -1.595
Income (BRL)              0.0081      0.002      3.812      0.000       0.004       0.012
==============================================================================
Omnibus:                     2158.208   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            35014.909
Skew:                           1.438   Prob(JB):                         0.00
Kurtosis:                      15.014   Cond. No.                     2.05e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [335]:
# Doses aplicadas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model1b = sm.OLS(y1, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1b)

model1b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.468
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     325.8
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                -1228.0
No. Observations:                5507   AIC:                             2494.
Df Residuals:                    5488   BIC:                             2620.
Df Model:                          18                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.6070      0.697     -2.307      0.021      -2.972      -0.242
Far-left 18               3.7817      2.003      1.888      0.059      -0.144       7.707
Far-right 22             -1.2700      1.228     -1.034      0.301      -3.677       1.137
elderly(>70yo)            0.0486      0.004     13.489      0.000       0.042       0.056
Rural pop.            -3.707e-05      0.000     -0.131      0.896      -0.001       0.001
White                    -0.0225      0.005     -4.955      0.000      -0.031      -0.014
Black                    -0.0238      0.005     -4.925      0.000      -0.033      -0.014
Brown                    -0.0275      0.005     -6.004      0.000      -0.037      -0.019
Indigenous               -0.0261      0.005     -5.637      0.000      -0.035      -0.017
Pop. density          -7.855e-06   1.12e-05     -0.702      0.483   -2.98e-05    1.41e-05
Catholic                  1.0280      0.070     14.662      0.000       0.891       1.165
Evangelic                 0.5824      0.108      5.371      0.000       0.370       0.795
Pentecostal Evangelic     0.9667      0.100      9.700      0.000       0.771       1.162
No religion               1.1365      0.143      7.964      0.000       0.857       1.416
Education 8y less         0.0415      0.006      7.411      0.000       0.031       0.053
Education 8-10y           0.0467      0.006      7.947      0.000       0.035       0.058
Education 11-14y          0.0511      0.006      8.951      0.000       0.040       0.062
Education 15y plus        0.0151      0.007      2.153      0.031       0.001       0.029
Income (BRL)              0.0001   1.71e-05      8.450      0.000       0.000       0.000
==============================================================================
Omnibus:                     3561.765   Durbin-Watson:                   1.449
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           551491.155
Skew:                           2.108   Prob(JB):                         0.00
Kurtosis:                      51.843   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [336]:
# Doses recebidas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2b = sm.OLS(y2, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2b)

model2b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     313.8
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                -321.51
No. Observations:                5507   AIC:                             681.0
Df Residuals:                    5488   BIC:                             806.7
Df Model:                          18                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.2379      0.500      4.475      0.000       1.258       3.218
Far-left 18               1.6976      1.741      0.975      0.329      -1.714       5.109
Far-right 22             -2.8103      1.256     -2.237      0.025      -5.272      -0.348
elderly(>70yo)            0.0660      0.004     15.224      0.000       0.057       0.074
Rural pop.               -0.0013      0.000     -4.770      0.000      -0.002      -0.001
White                    -0.0195      0.004     -4.864      0.000      -0.027      -0.012
Black                    -0.0242      0.004     -5.639      0.000      -0.033      -0.016
Brown                    -0.0237      0.004     -5.851      0.000      -0.032      -0.016
Indigenous               -0.0226      0.004     -5.341      0.000      -0.031      -0.014
Pop. density          -4.055e-06   6.84e-06     -0.593      0.553   -1.75e-05    9.35e-06
Catholic                  1.2049      0.118     10.184      0.000       0.973       1.437
Evangelic                 0.6339      0.146      4.356      0.000       0.349       0.919
Pentecostal Evangelic     1.0204      0.175      5.823      0.000       0.677       1.364
No religion               1.0808      0.168      6.418      0.000       0.751       1.411
Education 8y less        -0.0016      0.003     -0.453      0.650      -0.008       0.005
Education 8-10y           0.0084      0.004      2.160      0.031       0.001       0.016
Education 11-14y          0.0060      0.004      1.525      0.127      -0.002       0.014
Education 15y plus       -0.0267      0.005     -4.957      0.000      -0.037      -0.016
Income (BRL)              0.0001   1.65e-05      7.171      0.000    8.58e-05       0.000
==============================================================================
Omnibus:                      742.384   Durbin-Watson:                   1.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3508.212
Skew:                           0.572   Prob(JB):                         0.00
Kurtosis:                       6.739   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [337]:
# 3ª dose aplicada ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model3b = sm.OLS(y3, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3b)

model3b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.455
Model:                            OLS   Adj. R-squared:                  0.453
Method:                 Least Squares   F-statistic:                     251.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                 7556.3
No. Observations:                5507   AIC:                        -1.507e+04
Df Residuals:                    5488   BIC:                        -1.495e+04
Df Model:                          18                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.4952      0.120     -4.113      0.000      -0.731      -0.259
Far-left 18              -0.1661      0.447     -0.371      0.710      -1.043       0.711
Far-right 22             -2.2399      0.333     -6.731      0.000      -2.892      -1.588
elderly(>70yo)            0.0178      0.001     22.111      0.000       0.016       0.019
Rural pop.                0.0002   6.12e-05      3.347      0.001    8.48e-05       0.000
White                    -0.0003      0.001     -0.274      0.784      -0.002       0.002
Black                    -0.0013      0.001     -1.263      0.207      -0.003       0.001
Brown                    -0.0012      0.001     -1.230      0.219      -0.003       0.001
Indigenous               -0.0004      0.001     -0.391      0.696      -0.002       0.002
Pop. density          -1.518e-07   1.31e-06     -0.116      0.908   -2.72e-06    2.42e-06
Catholic                  0.0901      0.013      6.761      0.000       0.064       0.116
Evangelic                 0.0006      0.027      0.022      0.982      -0.052       0.054
Pentecostal Evangelic     0.0960      0.027      3.605      0.000       0.044       0.148
No religion               0.0878      0.027      3.292      0.001       0.036       0.140
Education 8y less         0.0048      0.001      5.369      0.000       0.003       0.007
Education 8-10y           0.0063      0.001      5.973      0.000       0.004       0.008
Education 11-14y          0.0063      0.001      6.592      0.000       0.004       0.008
Education 15y plus        0.0008      0.001      0.567      0.571      -0.002       0.004
Income (BRL)           2.631e-05   4.39e-06      5.987      0.000    1.77e-05    3.49e-05
==============================================================================
Omnibus:                     2106.284   Durbin-Watson:                   1.794
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23791.506
Skew:                           1.503   Prob(JB):                         0.00
Kurtosis:                      12.729   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [338]:
# 3ª dose recebida ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model4b = sm.OLS(y4, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4b)

model4b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     292.6
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:15   Log-Likelihood:                 8028.6
No. Observations:                5507   AIC:                        -1.602e+04
Df Residuals:                    5488   BIC:                        -1.589e+04
Df Model:                          18                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3427      0.105     -3.261      0.001      -0.549      -0.137
Far-left 18              -0.1351      0.398     -0.339      0.734      -0.915       0.645
Far-right 22             -2.1665      0.309     -7.015      0.000      -2.772      -1.561
elderly(>70yo)            0.0179      0.001     23.607      0.000       0.016       0.019
Rural pop.                0.0001   5.48e-05      1.851      0.064   -5.96e-06       0.000
White                    -0.0004      0.001     -0.396      0.692      -0.002       0.001
Black                    -0.0015      0.001     -1.595      0.111      -0.003       0.000
Brown                    -0.0013      0.001     -1.463      0.144      -0.003       0.000
Indigenous               -0.0004      0.001     -0.463      0.643      -0.002       0.001
Pop. density           1.861e-07   1.19e-06      0.156      0.876   -2.15e-06    2.52e-06
Catholic                  0.1026      0.014      7.209      0.000       0.075       0.130
Evangelic                 0.0101      0.026      0.392      0.695      -0.041       0.061
Pentecostal Evangelic     0.1019      0.028      3.660      0.000       0.047       0.156
No religion               0.0896      0.025      3.533      0.000       0.040       0.139
Education 8y less         0.0033      0.001      4.547      0.000       0.002       0.005
Education 8-10y           0.0049      0.001      5.861      0.000       0.003       0.006
Education 11-14y          0.0049      0.001      5.903      0.000       0.003       0.006
Education 15y plus       -0.0009      0.001     -0.725      0.469      -0.003       0.002
Income (BRL)           2.541e-05   3.89e-06      6.524      0.000    1.78e-05     3.3e-05
==============================================================================
Omnibus:                     1523.344   Durbin-Watson:                   1.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9808.466
Skew:                           1.163   Prob(JB):                         0.00
Kurtosis:                       9.110   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [339]:
# Cobertura vacinal geral ~ Far-left '18, Far-right '22

# Create the OLS model with a different name
model5b = sm.OLS(y5, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5b)

model5b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.194
Method:                 Least Squares   F-statistic:                     62.93
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.47e-207
Time:                        13:55:15   Log-Likelihood:                -24756.
No. Observations:                5507   AIC:                         4.955e+04
Df Residuals:                    5488   BIC:                         4.968e+04
Df Model:                          18                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   169.1307     41.816      4.045      0.000      87.173     251.088
Far-left 18            -744.2841    135.364     -5.498      0.000   -1009.593    -478.975
Far-right 22            659.3257    111.274      5.925      0.000     441.232     877.419
elderly(>70yo)            2.1229      0.261      8.128      0.000       1.611       2.635
Rural pop.               -0.0024      0.021     -0.117      0.907      -0.043       0.038
White                    -1.3221      0.339     -3.904      0.000      -1.986      -0.658
Black                    -1.5721      0.360     -4.371      0.000      -2.277      -0.867
Brown                    -1.4762      0.341     -4.332      0.000      -2.144      -0.808
Indigenous               -1.5821      0.343     -4.607      0.000      -2.255      -0.909
Pop. density             -0.0014      0.001     -2.197      0.028      -0.003      -0.000
Catholic                  9.0280      3.098      2.914      0.004       2.956      15.100
Evangelic                -1.8591      7.904     -0.235      0.814     -17.351      13.633
Pentecostal Evangelic    16.8546      6.258      2.693      0.007       4.590      29.119
No religion             -44.5360      9.594     -4.642      0.000     -63.340     -25.732
Education 8y less         0.2174      0.280      0.777      0.437      -0.331       0.766
Education 8-10y           0.3644      0.315      1.158      0.247      -0.253       0.981
Education 11-14y         -0.2457      0.310     -0.792      0.428      -0.854       0.362
Education 15y plus       -2.9741      0.477     -6.237      0.000      -3.909      -2.039
Income (BRL)              0.0116      0.001      7.833      0.000       0.009       0.014
==============================================================================
Omnibus:                      525.470   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2037.095
Skew:                           0.421   Prob(JB):                         0.00
Kurtosis:                       5.858   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [340]:
# Cobertura contra Polio ~ Far-left '18, Far-right '22

# Create the OLS model with a different name
model6b = sm.OLS(y6, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6b)

model6b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.128
Method:                 Least Squares   F-statistic:                     36.77
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          3.05e-121
Time:                        13:55:15   Log-Likelihood:                -26149.
No. Observations:                5507   AIC:                         5.234e+04
Df Residuals:                    5488   BIC:                         5.246e+04
Df Model:                          18                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   214.8381     53.517      4.014      0.000     109.946     319.730
Far-left 18            -800.4412    172.286     -4.646      0.000   -1138.115    -462.768
Far-right 22            786.0309    151.826      5.177      0.000     488.457    1083.605
elderly(>70yo)            2.5707      0.356      7.224      0.000       1.873       3.268
Rural pop.                0.0006      0.026      0.021      0.983      -0.050       0.051
White                    -1.4203      0.438     -3.241      0.001      -2.279      -0.561
Black                    -1.7297      0.463     -3.734      0.000      -2.637      -0.822
Brown                    -1.5882      0.440     -3.609      0.000      -2.451      -0.726
Indigenous               -1.8109      0.441     -4.107      0.000      -2.675      -0.947
Pop. density             -0.0010      0.001     -1.239      0.215      -0.002       0.001
Catholic                  9.8168      3.896      2.520      0.012       2.181      17.452
Evangelic               -11.2182     12.838     -0.874      0.382     -36.381      13.945
Pentecostal Evangelic    13.4050      8.342      1.607      0.108      -2.945      29.755
No religion             -31.6006     12.358     -2.557      0.011     -55.822      -7.379
Education 8y less        -0.0016      0.360     -0.004      0.996      -0.706       0.703
Education 8-10y           0.0546      0.406      0.134      0.893      -0.741       0.851
Education 11-14y         -0.4997      0.391     -1.277      0.202      -1.267       0.267
Education 15y plus       -3.0989      0.618     -5.015      0.000      -4.310      -1.888
Income (BRL)              0.0088      0.002      4.166      0.000       0.005       0.013
==============================================================================
Omnibus:                     2135.101   Durbin-Watson:                   1.946
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34095.200
Skew:                           1.422   Prob(JB):                         0.00
Kurtosis:                      14.853   Cond. No.                     7.34e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.34e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [341]:
models

In [342]:
# table1 = Stargazer([model1, model2, model3, model4, model5, model6])
# table1.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
# table2 = Stargazer([model1b, model2b, model3b, model4b, model5b, model6b ])
# table2.covariate_order(['Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])

In [343]:
table1 = Stargazer([model1, model2, model3, model4, model5, model6])
table1.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table1.show_degrees_of_freedom(False)
table2 = Stargazer([model1b, model2b, model3b, model4b, model5b, model6b ])
table2.covariate_order(['Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table2.show_degrees_of_freedom(False)

In [344]:
print(table1.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & -0.098$^{**}$ & -0.124$^{***}$ & -0.028$^{***}$ & -0.020$^{**}$ & 16.215$^{***}$ & 12.467$^{***}$ \\
  & (0.049) & (0.041) & (0.010) & (0.008) & (3.469) & (4.706) \\
 elderly(>70yo) & 0.047$^{***}$ & 0.065$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.417$^{***}$ & 2.860$^{***}$ \\
  & (0.004) & (0.004) & (0.001) & (0.001) & (0.264) & (0.361) \\
 Rural pop. & 0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & -0.016$^{}$ & -0.015$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.001$^{**}$ & -0.001$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.023$^{***}$ & -0.020$^{***}$ & -0.

In [345]:
print(table2.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Far-left 18 & 3.782$^{*}$ & 1.698$^{}$ & -0.166$^{}$ & -0.135$^{}$ & -744.284$^{***}$ & -800.441$^{***}$ \\
  & (2.003) & (1.741) & (0.447) & (0.398) & (135.364) & (172.286) \\
 Far-right 22 & -1.270$^{}$ & -2.810$^{**}$ & -2.240$^{***}$ & -2.167$^{***}$ & 659.326$^{***}$ & 786.031$^{***}$ \\
  & (1.228) & (1.256) & (0.333) & (0.309) & (111.274) & (151.826) \\
 elderly(>70yo) & 0.049$^{***}$ & 0.066$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.123$^{***}$ & 2.571$^{***}$ \\
  & (0.004) & (0.004) & (0.001) & (0.001) & (0.261) & (0.356) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{*}$ & -0.002$^{}$ & 0.001$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{

In [346]:
df['Region'].unique()

array(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'],
      dtype=object)

In [347]:
# setting DF for OLS
df_co = df[df['Region'] == 'Centro-Oeste']
df_ne = df[df['Region'] == 'Nordeste']
df_no = df[df['Region'] == 'Norte']
df_se = df[df['Region'] == 'Sudeste']
df_su = df[df['Region'] == 'Sul']

X2_co = sm.add_constant(df_co[list(x_vars1)])
X2_ne = sm.add_constant(df_ne[list(x_vars1)])
X2_no = sm.add_constant(df_no[list(x_vars1)])
X2_se = sm.add_constant(df_se[list(x_vars1)])
X2_su = sm.add_constant(df_su[list(x_vars1)])

# target2 (y2) as Doses Received
y2_co = df_co[y_vars[1]].values.reshape(-1, 1)
y2_ne = df_ne[y_vars[1]].values.reshape(-1, 1)
y2_no = df_no[y_vars[1]].values.reshape(-1, 1)
y2_se = df_se[y_vars[1]].values.reshape(-1, 1)
y2_su = df_su[y_vars[1]].values.reshape(-1, 1)

# target5 (y5) as General vaxx coverage
y5_co = df_co[y_vars[4]].values.reshape(-1, 1)
y5_ne = df_ne[y_vars[4]].values.reshape(-1, 1)
y5_no = df_no[y_vars[4]].values.reshape(-1, 1)
y5_se = df_se[y_vars[4]].values.reshape(-1, 1)
y5_su = df_su[y_vars[4]].values.reshape(-1, 1)

In [348]:
# Doses recebidas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2co = sm.OLS(y2_co, X2_co).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2co)

model2co.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     48.11
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           5.44e-88
Time:                        13:55:16   Log-Likelihood:                -14.182
No. Observations:                 446   AIC:                             64.36
Df Residuals:                     428   BIC:                             138.2
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.8949      3.134      1.881      0.060      -0.248      12.037
Bolsonaro Share 22       -0.6382      0.159     -4.010      0.000      -0.950      -0.326
elderly(>70yo)            0.0152      0.012      1.229      0.219      -0.009       0.040
Rural pop.            -7.626e-05      0.001     -0.075      0.940      -0.002       0.002
White                    -0.0265      0.014     -1.832      0.067      -0.055       0.002
Black                    -0.0529      0.016     -3.383      0.001      -0.084      -0.022
Brown                    -0.0341      0.014     -2.386      0.017      -0.062      -0.006
Indigenous               -0.0327      0.014     -2.283      0.022      -0.061      -0.005
Pop. density             -0.0002   5.54e-05     -3.305      0.001      -0.000   -7.45e-05
Catholic                  1.4086      0.102     13.841      0.000       1.209       1.608
Evangelic                -0.4809      0.501     -0.959      0.337      -1.463       0.501
Pentecostal Evangelic     1.8248      0.172     10.638      0.000       1.489       2.161
No religion               1.0101      0.347      2.913      0.004       0.330       1.690
Education 8y less        -0.0264      0.029     -0.922      0.356      -0.083       0.030
Education 8-10y          -0.0184      0.030     -0.625      0.532      -0.076       0.039
Education 11-14y         -0.0096      0.029     -0.333      0.739      -0.066       0.047
Education 15y plus       -0.0563      0.035     -1.607      0.108      -0.125       0.012
Income (BRL)              0.0002   6.93e-05      3.531      0.000       0.000       0.000
==============================================================================
Omnibus:                       37.996   Durbin-Watson:                   2.095
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               59.788
Skew:                           0.581   Prob(JB):                     1.04e-13
Kurtosis:                       4.367   Cond. No.                     4.00e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large,  4e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [349]:
# Create the OLS model with a different name
model2ne = sm.OLS(y2_ne, X2_ne).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2ne)

model2ne.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     64.71
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.27e-171
Time:                        13:55:16   Log-Likelihood:                -35.960
No. Observations:                1787   AIC:                             107.9
Df Residuals:                    1769   BIC:                             206.7
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.3964      0.689      3.479      0.001       1.046       3.746
Bolsonaro Share 22       -0.3978      0.092     -4.323      0.000      -0.578      -0.217
elderly(>70yo)            0.0592      0.007      8.680      0.000       0.046       0.073
Rural pop.               -0.0011      0.000     -2.688      0.007      -0.002      -0.000
White                    -0.0135      0.006     -2.450      0.014      -0.024      -0.003
Black                    -0.0163      0.006     -2.758      0.006      -0.028      -0.005
Brown                    -0.0198      0.006     -3.536      0.000      -0.031      -0.009
Indigenous               -0.0150      0.006     -2.599      0.009      -0.026      -0.004
Pop. density          -2.603e-05   9.58e-06     -2.717      0.007   -4.48e-05   -7.25e-06
Catholic                  1.5246      0.218      6.999      0.000       1.098       1.952
Evangelic                 0.1005      0.426      0.236      0.813      -0.734       0.935
Pentecostal Evangelic     1.0281      0.260      3.954      0.000       0.518       1.538
No religion               1.6493      0.240      6.875      0.000       1.179       2.119
Education 8y less        -0.0109      0.004     -2.616      0.009      -0.019      -0.003
Education 8-10y           0.0046      0.005      0.857      0.392      -0.006       0.015
Education 11-14y         -0.0128      0.006     -2.265      0.024      -0.024      -0.002
Education 15y plus       -0.0052      0.011     -0.455      0.649      -0.028       0.017
Income (BRL)              0.0003   4.64e-05      5.680      0.000       0.000       0.000
==============================================================================
Omnibus:                      161.025   Durbin-Watson:                   1.861
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              848.201
Skew:                           0.234   Prob(JB):                    6.54e-185
Kurtosis:                       6.342   Cond. No.                     1.04e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [350]:
# Create the OLS model with a different name
model2no = sm.OLS(y2_no, X2_no).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2no)

model2no.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.558
Method:                 Least Squares   F-statistic:                     37.51
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           5.70e-74
Time:                        13:55:16   Log-Likelihood:                 26.497
No. Observations:                 449   AIC:                            -16.99
Df Residuals:                     431   BIC:                             56.93
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.3699      1.445      0.256      0.798      -2.461       3.201
Bolsonaro Share 22       -0.1404      0.096     -1.464      0.143      -0.328       0.048
elderly(>70yo)            0.0912      0.014      6.655      0.000       0.064       0.118
Rural pop.                0.0015      0.001      1.682      0.093      -0.000       0.003
White                     0.0039      0.011      0.364      0.716      -0.017       0.025
Black                    -0.0116      0.012     -1.002      0.316      -0.034       0.011
Brown                    -0.0070      0.011     -0.651      0.515      -0.028       0.014
Indigenous               -0.0050      0.011     -0.459      0.646      -0.026       0.016
Pop. density          -8.292e-05   3.96e-05     -2.096      0.036      -0.000   -5.37e-06
Catholic                  0.9070      0.104      8.752      0.000       0.704       1.110
Evangelic                 0.1322      0.484      0.273      0.785      -0.817       1.082
Pentecostal Evangelic     0.2794      0.205      1.366      0.172      -0.121       0.680
No religion               0.6448      0.409      1.578      0.115      -0.156       1.446
Education 8y less        -0.0010      0.011     -0.093      0.926      -0.023       0.021
Education 8-10y           0.0174      0.013      1.291      0.197      -0.009       0.044
Education 11-14y         -0.0029      0.012     -0.239      0.811      -0.027       0.021
Education 15y plus        0.0384      0.026      1.476      0.140      -0.013       0.089
Income (BRL)              0.0002   8.11e-05      2.610      0.009    5.27e-05       0.000
==============================================================================
Omnibus:                       81.923   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              267.107
Skew:                           0.818   Prob(JB):                     9.97e-59
Kurtosis:                       6.406   Cond. No.                     1.97e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [351]:
# Create the OLS model with a different name
model2se = sm.OLS(y2_se, X2_se).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2se)

model2se.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.384
Model:                            OLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     57.67
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          2.97e-153
Time:                        13:55:16   Log-Likelihood:                -79.092
No. Observations:                1666   AIC:                             194.2
Df Residuals:                    1648   BIC:                             291.7
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.8318      1.332      2.126      0.034       0.221       5.443
Bolsonaro Share 22       -0.0535      0.087     -0.615      0.539      -0.224       0.117
elderly(>70yo)            0.0561      0.006      9.814      0.000       0.045       0.067
Rural pop.               -0.0003      0.001     -0.592      0.554      -0.001       0.001
White                    -0.0148      0.009     -1.667      0.096      -0.032       0.003
Black                    -0.0250      0.009     -2.667      0.008      -0.043      -0.007
Brown                    -0.0202      0.009     -2.236      0.025      -0.038      -0.002
Indigenous               -0.0182      0.009     -2.013      0.044      -0.036      -0.000
Pop. density           -3.17e-06    8.1e-06     -0.391      0.696    -1.9e-05    1.27e-05
Catholic                  1.1144      0.115      9.670      0.000       0.889       1.340
Evangelic                 0.8186      0.195      4.196      0.000       0.436       1.201
Pentecostal Evangelic     1.0299      0.182      5.644      0.000       0.672       1.388
No religion               0.9620      0.289      3.329      0.001       0.396       1.528
Education 8y less        -0.0101      0.009     -1.073      0.283      -0.029       0.008
Education 8-10y           0.0009      0.010      0.091      0.928      -0.019       0.021
Education 11-14y         -0.0033      0.010     -0.331      0.741      -0.023       0.016
Education 15y plus       -0.0619      0.012     -5.370      0.000      -0.085      -0.039
Income (BRL)              0.0002   3.18e-05      5.431      0.000       0.000       0.000
==============================================================================
Omnibus:                      224.872   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              551.018
Skew:                           0.754   Prob(JB):                    2.23e-120
Kurtosis:                       5.380   Cond. No.                     3.65e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 3.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [352]:
# Create the OLS model with a different name
model2su = sm.OLS(y2_su, X2_su).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2su)

model2su.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     50.46
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          7.88e-126
Time:                        13:55:16   Log-Likelihood:                 159.51
No. Observations:                1159   AIC:                            -283.0
Df Residuals:                    1141   BIC:                            -192.0
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.7986      1.344     -0.594      0.552      -3.433       1.836
Bolsonaro Share 22       -0.0804      0.081     -0.997      0.319      -0.238       0.078
elderly(>70yo)            0.0402      0.006      7.050      0.000       0.029       0.051
Rural pop.               -0.0001      0.000     -0.284      0.777      -0.001       0.001
White                    -0.0015      0.009     -0.175      0.861      -0.018       0.015
Black                     0.0022      0.009      0.239      0.811      -0.016       0.021
Brown                    -0.0052      0.009     -0.598      0.550      -0.022       0.012
Indigenous               -0.0041      0.009     -0.451      0.652      -0.022       0.014
Pop. density          -1.682e-05   1.43e-05     -1.177      0.239   -4.48e-05    1.12e-05
Catholic                  1.4089      0.102     13.824      0.000       1.209       1.609
Evangelic                 1.2601      0.130      9.673      0.000       1.005       1.515
Pentecostal Evangelic     1.5374      0.199      7.718      0.000       1.147       1.928
No religion               0.9942      0.474      2.097      0.036       0.065       1.923
Education 8y less         0.0091      0.010      0.887      0.375      -0.011       0.029
Education 8-10y           0.0203      0.011      1.888      0.059      -0.001       0.041
Education 11-14y          0.0127      0.011      1.158      0.247      -0.009       0.034
Education 15y plus       -0.0054      0.012     -0.464      0.643      -0.028       0.018
Income (BRL)              0.0001   2.52e-05      4.882      0.000    7.38e-05       0.000
==============================================================================
Omnibus:                       53.190   Durbin-Watson:                   1.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.054
Skew:                           0.362   Prob(JB):                     4.59e-20
Kurtosis:                       4.149   Cond. No.                     3.79e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 3.79e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [353]:
table3 = Stargazer([model2co, model2ne, model2no, model2se, model2su ])
table3.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table3.show_degrees_of_freedom(False)
print(table3.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable:}} \
\cr \cline{5-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & -0.638$^{***}$ & -0.398$^{***}$ & -0.140$^{}$ & -0.053$^{}$ & -0.080$^{}$ \\
  & (0.159) & (0.092) & (0.096) & (0.087) & (0.081) \\
 elderly(>70yo) & 0.015$^{}$ & 0.059$^{***}$ & 0.091$^{***}$ & 0.056$^{***}$ & 0.040$^{***}$ \\
  & (0.012) & (0.007) & (0.014) & (0.006) & (0.006) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.001$^{*}$ & -0.000$^{}$ & -0.000$^{}$ \\
  & (0.001) & (0.000) & (0.001) & (0.001) & (0.000) \\
 Pop. density & -0.000$^{***}$ & -0.000$^{***}$ & -0.000$^{**}$ & -0.000$^{}$ & -0.000$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.000) \\
 White & -0.027$^{*}$ & -0.013$^{**}$ & 0.004$^{}$ & -0.015$^{*}$ & -0.002$^{}$ \\
  & (0.014) & (0.006) & (0.011) & (0.009) & (0.009) \\
 Black & -0.053$^{***}

In [354]:
# Vaxx coverage ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2co = sm.OLS(y5_co, X2_co).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2co)

model2co.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     5.007
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           5.84e-10
Time:                        13:55:16   Log-Likelihood:                -2007.4
No. Observations:                 446   AIC:                             4051.
Df Residuals:                     428   BIC:                             4125.
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -310.5178    263.339     -1.179      0.238    -826.653     205.618
Bolsonaro Share 22       22.8481     15.463      1.478      0.140      -7.458      53.154
elderly(>70yo)            1.2090      1.062      1.138      0.255      -0.873       3.291
Rural pop.                0.1179      0.083      1.426      0.154      -0.044       0.280
White                     0.1396      1.194      0.117      0.907      -2.200       2.479
Black                    -0.6896      1.358     -0.508      0.612      -3.352       1.973
Brown                    -0.0467      1.162     -0.040      0.968      -2.324       2.231
Indigenous               -0.4197      1.156     -0.363      0.717      -2.686       1.847
Pop. density             -0.0038      0.003     -1.315      0.189      -0.009       0.002
Catholic                 11.8801      8.691      1.367      0.172      -5.153      28.913
Evangelic               -22.1922     51.583     -0.430      0.667    -123.293      78.908
Pentecostal Evangelic    -5.4700     16.816     -0.325      0.745     -38.429      27.489
No religion              18.1826     30.968      0.587      0.557     -42.513      78.878
Education 8y less         3.7188      2.380      1.562      0.118      -0.947       8.384
Education 8-10y           3.5320      2.434      1.451      0.147      -1.238       8.302
Education 11-14y          3.6968      2.399      1.541      0.123      -1.005       8.398
Education 15y plus        3.2565      2.918      1.116      0.264      -2.463       8.975
Income (BRL)             -0.0016      0.005     -0.311      0.756      -0.012       0.009
==============================================================================
Omnibus:                       37.747   Durbin-Watson:                   2.069
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.519
Skew:                           0.527   Prob(JB):                     1.32e-15
Kurtosis:                       4.605   Cond. No.                     4.00e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large,  4e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [355]:
# Create the OLS model with a different name
model2ne = sm.OLS(y5_ne, X2_ne).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2ne)

model2ne.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     16.67
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           2.93e-46
Time:                        13:55:16   Log-Likelihood:                -7939.3
No. Observations:                1787   AIC:                         1.591e+04
Df Residuals:                    1769   BIC:                         1.601e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   317.7456     56.448      5.629      0.000     207.111     428.381
Bolsonaro Share 22      -14.5672      7.975     -1.827      0.068     -30.199       1.064
elderly(>70yo)            2.1491      0.442      4.865      0.000       1.283       3.015
Rural pop.                0.0043      0.036      0.119      0.905      -0.066       0.075
White                    -1.7548      0.494     -3.556      0.000      -2.722      -0.788
Black                    -2.1315      0.533     -3.999      0.000      -3.176      -1.087
Brown                    -2.0317      0.497     -4.091      0.000      -3.005      -1.058
Indigenous               -2.2472      0.507     -4.433      0.000      -3.241      -1.254
Pop. density             -0.0024      0.002     -1.409      0.159      -0.006       0.001
Catholic                  8.6236      5.369      1.606      0.108      -1.899      19.146
Evangelic                19.2486     31.490      0.611      0.541     -42.471      80.968
Pentecostal Evangelic   -15.4642     15.765     -0.981      0.327     -46.363      15.435
No religion              -9.5345     16.175     -0.589      0.556     -41.237      22.168
Education 8y less        -0.7741      0.352     -2.197      0.028      -1.465      -0.084
Education 8-10y          -0.6684      0.500     -1.337      0.181      -1.648       0.311
Education 11-14y         -1.4434      0.472     -3.061      0.002      -2.368      -0.519
Education 15y plus       -1.2904      1.218     -1.059      0.289      -3.678       1.097
Income (BRL)              0.0112      0.005      2.423      0.015       0.002       0.020
==============================================================================
Omnibus:                      227.461   Durbin-Watson:                   1.924
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              781.199
Skew:                           0.613   Prob(JB):                    2.32e-170
Kurtosis:                       5.998   Cond. No.                     1.04e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [356]:
# Create the OLS model with a different name
model2no = sm.OLS(y5_no, X2_no).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2no)

model2no.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     12.89
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           2.67e-29
Time:                        13:55:16   Log-Likelihood:                -1967.3
No. Observations:                 449   AIC:                             3971.
Df Residuals:                     431   BIC:                             4044.
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -23.4738    148.881     -0.158      0.875    -315.276     268.328
Bolsonaro Share 22        2.9541      8.986      0.329      0.742     -14.659      20.567
elderly(>70yo)            4.5742      1.257      3.640      0.000       2.111       7.037
Rural pop.               -0.1632      0.068     -2.404      0.016      -0.296      -0.030
White                    -1.4902      1.146     -1.300      0.193      -3.736       0.756
Black                    -1.7764      1.183     -1.502      0.133      -4.094       0.542
Brown                    -2.0649      1.127     -1.833      0.067      -4.273       0.143
Indigenous               -2.1138      1.132     -1.867      0.062      -4.333       0.105
Pop. density             -0.0145      0.003     -4.401      0.000      -0.021      -0.008
Catholic                  8.7390      7.368      1.186      0.236      -5.702      23.180
Evangelic                -5.3640     32.068     -0.167      0.867     -68.216      57.488
Pentecostal Evangelic    -5.1556     15.604     -0.330      0.741     -35.738      25.427
No religion             -19.9880     24.950     -0.801      0.423     -68.889      28.913
Education 8y less         2.6943      1.058      2.547      0.011       0.621       4.767
Education 8-10y           2.1353      1.208      1.768      0.077      -0.232       4.502
Education 11-14y          2.8452      1.140      2.496      0.013       0.611       5.080
Education 15y plus       -1.5717      2.570     -0.612      0.541      -6.608       3.465
Income (BRL)              0.0085      0.006      1.330      0.184      -0.004       0.021
==============================================================================
Omnibus:                        0.861   Durbin-Watson:                   1.846
Prob(Omnibus):                  0.650   Jarque-Bera (JB):                0.655
Skew:                           0.049   Prob(JB):                        0.721
Kurtosis:                       3.160   Cond. No.                     1.97e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.97e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [357]:
# Create the OLS model with a different name
model2se = sm.OLS(y5_se, X2_se).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2se)

model2se.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     16.28
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           7.73e-45
Time:                        13:55:16   Log-Likelihood:                -7516.9
No. Observations:                1666   AIC:                         1.507e+04
Df Residuals:                    1648   BIC:                         1.517e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   386.6478    110.562      3.497      0.000     169.950     603.346
Bolsonaro Share 22        5.7975      9.019      0.643      0.520     -11.880      23.475
elderly(>70yo)            1.8772      0.539      3.481      0.000       0.820       2.934
Rural pop.               -0.0508      0.047     -1.070      0.285      -0.144       0.042
White                    -1.0757      0.606     -1.775      0.076      -2.264       0.112
Black                    -1.3642      0.626     -2.179      0.029      -2.591      -0.137
Brown                    -1.2622      0.615     -2.054      0.040      -2.467      -0.058
Indigenous               -1.0606      0.610     -1.738      0.082      -2.257       0.136
Pop. density             -0.0002      0.001     -0.333      0.739      -0.002       0.001
Catholic                  3.5669      8.589      0.415      0.678     -13.267      20.400
Evangelic                19.0762     15.424      1.237      0.216     -11.153      49.306
Pentecostal Evangelic     2.9726     15.306      0.194      0.846     -27.027      32.972
No religion            -109.2553     28.447     -3.841      0.000    -165.011     -53.500
Education 8y less        -1.9104      0.919     -2.078      0.038      -3.712      -0.109
Education 8-10y          -2.1305      0.968     -2.202      0.028      -4.027      -0.234
Education 11-14y         -2.8761      0.948     -3.034      0.002      -4.734      -1.018
Education 15y plus       -4.1500      1.036     -4.006      0.000      -6.180      -2.120
Income (BRL)              0.0047      0.003      1.787      0.074      -0.000       0.010
==============================================================================
Omnibus:                       84.036   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              290.875
Skew:                           0.078   Prob(JB):                     6.87e-64
Kurtosis:                       5.041   Cond. No.                     3.65e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 3.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [358]:
# Create the OLS model with a different name
model2su = sm.OLS(y5_su, X2_su).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2su)

model2su.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.176
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     10.83
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.25e-27
Time:                        13:55:16   Log-Likelihood:                -5188.1
No. Observations:                1159   AIC:                         1.041e+04
Df Residuals:                    1141   BIC:                         1.050e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   179.0237    123.487      1.450      0.147     -63.006     421.053
Bolsonaro Share 22       -8.1196      9.207     -0.882      0.378     -26.165       9.926
elderly(>70yo)            1.9489      0.569      3.424      0.001       0.833       3.064
Rural pop.                0.1276      0.046      2.762      0.006       0.037       0.218
White                     0.8419      0.836      1.007      0.314      -0.797       2.481
Black                     0.3773      0.940      0.401      0.688      -1.466       2.220
Brown                     0.7428      0.863      0.861      0.389      -0.949       2.434
Indigenous                0.4565      0.877      0.521      0.603      -1.262       2.175
Pop. density             -0.0039      0.002     -2.075      0.038      -0.008      -0.000
Catholic                  1.4072      7.892      0.178      0.858     -14.060      16.875
Evangelic               -13.0265     13.616     -0.957      0.339     -39.713      13.660
Pentecostal Evangelic     4.6271     15.519      0.298      0.766     -25.789      35.043
No religion            -104.7471     37.442     -2.798      0.005    -178.133     -31.361
Education 8y less        -2.0229      0.890     -2.272      0.023      -3.768      -0.278
Education 8-10y          -2.0968      0.991     -2.115      0.034      -4.040      -0.154
Education 11-14y         -1.9184      0.924     -2.077      0.038      -3.729      -0.108
Education 15y plus       -4.1572      1.112     -3.740      0.000      -6.336      -1.979
Income (BRL)              0.0122      0.003      3.973      0.000       0.006       0.018
==============================================================================
Omnibus:                      243.439   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1841.263
Skew:                           0.754   Prob(JB):                         0.00
Kurtosis:                       8.988   Cond. No.                     3.79e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 3.79e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [359]:
table4 = Stargazer([model2co, model2ne, model2no, model2se, model2su])
table4.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table4.show_degrees_of_freedom(False)
print(table4.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable:}} \
\cr \cline{5-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & 22.848$^{}$ & -14.567$^{*}$ & 2.954$^{}$ & 5.798$^{}$ & -8.120$^{}$ \\
  & (15.463) & (7.975) & (8.986) & (9.019) & (9.207) \\
 elderly(>70yo) & 1.209$^{}$ & 2.149$^{***}$ & 4.574$^{***}$ & 1.877$^{***}$ & 1.949$^{***}$ \\
  & (1.062) & (0.442) & (1.257) & (0.539) & (0.569) \\
 Rural pop. & 0.118$^{}$ & 0.004$^{}$ & -0.163$^{**}$ & -0.051$^{}$ & 0.128$^{***}$ \\
  & (0.083) & (0.036) & (0.068) & (0.047) & (0.046) \\
 Pop. density & -0.004$^{}$ & -0.002$^{}$ & -0.015$^{***}$ & -0.000$^{}$ & -0.004$^{**}$ \\
  & (0.003) & (0.002) & (0.003) & (0.001) & (0.002) \\
 White & 0.140$^{}$ & -1.755$^{***}$ & -1.490$^{}$ & -1.076$^{*}$ & 0.842$^{}$ \\
  & (1.194) & (0.494) & (1.146) & (0.606) & (0.836) \\
 Black & -0.690$^{}$ & -2.132$^{

------------------------
Sensitivity tests

In [360]:
# Doses aplicadas ~ Ciro '18

# Create the OLS model with a different name
model1 = sm.OLS(y1, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     356.4
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:16   Log-Likelihood:                -990.64
No. Observations:                5507   AIC:                             2017.
Df Residuals:                    5489   BIC:                             2136.
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.7846      0.607      1.294      0.196      -0.404       1.973
Ciro Share 18            -1.1429      0.088    -12.960      0.000      -1.316      -0.970
elderly(>70yo)            0.0572      0.003     16.589      0.000       0.050       0.064
Rural pop.               -0.0008      0.000     -2.907      0.004      -0.001      -0.000
White                    -0.0203      0.004     -4.602      0.000      -0.029      -0.012
Black                    -0.0237      0.005     -5.020      0.000      -0.033      -0.014
Brown                    -0.0250      0.004     -5.608      0.000      -0.034      -0.016
Indigenous               -0.0242      0.004     -5.392      0.000      -0.033      -0.015
Pop. density           1.539e-05   1.25e-05      1.227      0.220    -9.2e-06       4e-05
Catholic                  1.0022      0.065     15.492      0.000       0.875       1.129
Evangelic                 0.4469      0.107      4.187      0.000       0.238       0.656
Pentecostal Evangelic     0.7686      0.084      9.177      0.000       0.604       0.933
No religion               1.0442      0.137      7.624      0.000       0.776       1.313
Education 8y less         0.0173      0.005      3.697      0.000       0.008       0.026
Education 8-10y           0.0247      0.005      4.857      0.000       0.015       0.035
Education 11-14y          0.0256      0.005      5.345      0.000       0.016       0.035
Education 15y plus        0.0059      0.006      0.918      0.359      -0.007       0.018
Income (BRL)           8.055e-05   1.77e-05      4.560      0.000    4.59e-05       0.000
==============================================================================
Omnibus:                     4599.297   Durbin-Watson:                   1.640
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           857849.642
Skew:                           3.214   Prob(JB):                         0.00
Kurtosis:                      63.805   Cond. No.                     2.09e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [361]:
# Doses recebidas ~ Ciro '18

# Create the OLS model with a different name
model2 = sm.OLS(y2, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     331.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                -324.02
No. Observations:                5507   AIC:                             684.0
Df Residuals:                    5489   BIC:                             803.1
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.2878      0.513      4.459      0.000       1.282       3.293
Ciro Share 18             0.0349      0.063      0.559      0.576      -0.088       0.158
elderly(>70yo)            0.0652      0.005     14.413      0.000       0.056       0.074
Rural pop.               -0.0012      0.000     -4.619      0.000      -0.002      -0.001
White                    -0.0201      0.004     -4.984      0.000      -0.028      -0.012
Black                    -0.0248      0.004     -5.769      0.000      -0.033      -0.016
Brown                    -0.0242      0.004     -5.948      0.000      -0.032      -0.016
Indigenous               -0.0230      0.004     -5.394      0.000      -0.031      -0.015
Pop. density          -4.131e-06   7.02e-06     -0.588      0.556   -1.79e-05    9.63e-06
Catholic                  1.2043      0.119     10.084      0.000       0.970       1.438
Evangelic                 0.6369      0.149      4.278      0.000       0.345       0.929
Pentecostal Evangelic     1.0347      0.181      5.727      0.000       0.681       1.389
No religion               1.1222      0.169      6.632      0.000       0.791       1.454
Education 8y less        -0.0016      0.003     -0.482      0.630      -0.008       0.005
Education 8-10y           0.0083      0.004      2.196      0.028       0.001       0.016
Education 11-14y          0.0063      0.004      1.613      0.107      -0.001       0.014
Education 15y plus       -0.0274      0.005     -5.096      0.000      -0.038      -0.017
Income (BRL)              0.0001   1.72e-05      6.706      0.000    8.14e-05       0.000
==============================================================================
Omnibus:                      746.592   Durbin-Watson:                   1.829
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3595.733
Skew:                           0.570   Prob(JB):                         0.00
Kurtosis:                       6.791   Cond. No.                     2.09e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [362]:
# 3ª dose aplicada ~ Ciro '18

# Create the OLS model with a different name
model3 = sm.OLS(y3, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     259.7
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                 7530.8
No. Observations:                5507   AIC:                        -1.503e+04
Df Residuals:                    5489   BIC:                        -1.491e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3156      0.121     -2.613      0.009      -0.552      -0.079
Ciro Share 18            -0.0292      0.013     -2.210      0.027      -0.055      -0.003
elderly(>70yo)            0.0180      0.001     22.173      0.000       0.016       0.020
Rural pop.                0.0002   6.25e-05      3.338      0.001    8.62e-05       0.000
White                    -0.0008      0.001     -0.804      0.422      -0.003       0.001
Black                    -0.0021      0.001     -2.010      0.044      -0.004    -5.2e-05
Brown                    -0.0016      0.001     -1.671      0.095      -0.004       0.000
Indigenous               -0.0008      0.001     -0.844      0.398      -0.003       0.001
Pop. density          -4.777e-07   1.27e-06     -0.378      0.706   -2.96e-06       2e-06
Catholic                  0.0888      0.013      6.661      0.000       0.063       0.115
Evangelic                -0.0023      0.027     -0.083      0.933      -0.056       0.052
Pentecostal Evangelic     0.1017      0.027      3.761      0.000       0.049       0.155
No religion               0.1079      0.026      4.071      0.000       0.056       0.160
Education 8y less         0.0035      0.001      3.976      0.000       0.002       0.005
Education 8-10y           0.0048      0.001      4.707      0.000       0.003       0.007
Education 11-14y          0.0051      0.001      5.467      0.000       0.003       0.007
Education 15y plus       -0.0007      0.001     -0.478      0.633      -0.003       0.002
Income (BRL)           2.079e-05   4.51e-06      4.607      0.000    1.19e-05    2.96e-05
==============================================================================
Omnibus:                     2127.801   Durbin-Watson:                   1.792
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24362.277
Skew:                           1.518   Prob(JB):                         0.00
Kurtosis:                      12.846   Cond. No.                     2.09e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [363]:
# 3ª dose recebida ~ Ciro '18

# Create the OLS model with a different name
model4 = sm.OLS(y4, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.503
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     302.2
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                 7996.8
No. Observations:                5507   AIC:                        -1.596e+04
Df Residuals:                    5489   BIC:                        -1.584e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.2465      0.107     -2.305      0.021      -0.456      -0.037
Ciro Share 18             0.0089      0.014      0.640      0.522      -0.018       0.036
elderly(>70yo)            0.0178      0.001     23.122      0.000       0.016       0.019
Rural pop.                0.0001   5.53e-05      2.386      0.017    2.35e-05       0.000
White                    -0.0009      0.001     -1.026      0.305      -0.003       0.001
Black                    -0.0022      0.001     -2.376      0.018      -0.004      -0.000
Brown                    -0.0018      0.001     -2.010      0.044      -0.004   -4.47e-05
Indigenous               -0.0009      0.001     -0.989      0.323      -0.003       0.001
Pop. density          -7.711e-07   1.14e-06     -0.675      0.500   -3.01e-06    1.47e-06
Catholic                  0.1021      0.015      6.909      0.000       0.073       0.131
Evangelic                 0.0116      0.027      0.435      0.664      -0.041       0.064
Pentecostal Evangelic     0.1136      0.029      3.860      0.000       0.056       0.171
No religion               0.1131      0.025      4.438      0.000       0.063       0.163
Education 8y less         0.0028      0.001      3.979      0.000       0.001       0.004
Education 8-10y           0.0042      0.001      5.175      0.000       0.003       0.006
Education 11-14y          0.0046      0.001      5.745      0.000       0.003       0.006
Education 15y plus       -0.0020      0.001     -1.621      0.105      -0.004       0.000
Income (BRL)           2.206e-05   3.99e-06      5.534      0.000    1.42e-05    2.99e-05
==============================================================================
Omnibus:                     1507.600   Durbin-Watson:                   1.790
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9746.260
Skew:                           1.148   Prob(JB):                         0.00
Kurtosis:                       9.100   Cond. No.                     2.09e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [364]:
# Cobertura vacinal ~ Ciro '18

# Create the OLS model with a different name
model5 = sm.OLS(y5, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     62.73
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.36e-196
Time:                        13:55:17   Log-Likelihood:                -24785.
No. Observations:                5507   AIC:                         4.961e+04
Df Residuals:                    5489   BIC:                         4.972e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   155.9478     42.187      3.697      0.000      73.263     238.633
Ciro Share 18            -5.1677      4.363     -1.184      0.236     -13.719       3.384
elderly(>70yo)            2.3612      0.265      8.896      0.000       1.841       2.881
Rural pop.               -0.0179      0.021     -0.845      0.398      -0.059       0.024
White                    -1.2432      0.335     -3.715      0.000      -1.899      -0.587
Black                    -1.4804      0.354     -4.182      0.000      -2.174      -0.787
Brown                    -1.4183      0.337     -4.205      0.000      -2.079      -0.757
Indigenous               -1.5592      0.340     -4.590      0.000      -2.225      -0.893
Pop. density             -0.0018      0.001     -2.867      0.004      -0.003      -0.001
Catholic                  9.2792      3.106      2.987      0.003       3.191      15.367
Evangelic                -1.9016      7.966     -0.239      0.811     -17.515      13.712
Pentecostal Evangelic    14.9261      6.283      2.376      0.018       2.611      27.241
No religion             -55.4655      9.617     -5.767      0.000     -74.315     -36.616
Education 8y less         0.2843      0.291      0.978      0.328      -0.286       0.854
Education 8-10y           0.3831      0.323      1.187      0.235      -0.250       1.016
Education 11-14y         -0.2784      0.322     -0.864      0.388      -0.910       0.354
Education 15y plus       -2.8965      0.476     -6.081      0.000      -3.830      -1.963
Income (BRL)              0.0124      0.001      8.384      0.000       0.010       0.015
==============================================================================
Omnibus:                      528.666   Durbin-Watson:                   1.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2008.330
Skew:                           0.432   Prob(JB):                         0.00
Kurtosis:                       5.830   Cond. No.                     2.09e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [365]:
# Cobertura contra Polio ~ Ciro '18

# Create the OLS model with a different name
model6 = sm.OLS(y6, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     36.21
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          3.16e-113
Time:                        13:55:17   Log-Likelihood:                -26173.
No. Observations:                5507   AIC:                         5.238e+04
Df Residuals:                    5489   BIC:                         5.250e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   185.8932     54.648      3.402      0.001      78.785     293.002
Ciro Share 18            -0.3843      5.248     -0.073      0.942     -10.669       9.901
elderly(>70yo)            2.7870      0.362      7.692      0.000       2.077       3.497
Rural pop.               -0.0132      0.026     -0.504      0.615      -0.065       0.038
White                    -1.3256      0.434     -3.053      0.002      -2.177      -0.475
Black                    -1.6050      0.458     -3.506      0.000      -2.502      -0.708
Brown                    -1.5201      0.436     -3.485      0.000      -2.375      -0.665
Indigenous               -1.7773      0.437     -4.068      0.000      -2.634      -0.921
Pop. density             -0.0014      0.001     -1.891      0.059      -0.003    5.05e-05
Catholic                 10.2239      3.941      2.594      0.009       2.500      17.948
Evangelic               -10.6765     12.904     -0.827      0.408     -35.968      14.615
Pentecostal Evangelic    11.8580      8.333      1.423      0.155      -4.474      28.190
No religion             -43.6232     12.449     -3.504      0.000     -68.022     -19.224
Education 8y less         0.2034      0.380      0.535      0.592      -0.541       0.948
Education 8-10y           0.2049      0.424      0.484      0.629      -0.625       1.035
Education 11-14y         -0.4039      0.412     -0.980      0.327      -1.212       0.404
Education 15y plus       -2.9314      0.628     -4.670      0.000      -4.162      -1.701
Income (BRL)              0.0102      0.002      4.891      0.000       0.006       0.014
==============================================================================
Omnibus:                     2140.609   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34252.407
Skew:                           1.426   Prob(JB):                         0.00
Kurtosis:                      14.880   Cond. No.                     2.09e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.09e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [366]:
table5 = Stargazer([model1, model2, model3, model4, model5, model6])
table5.covariate_order(['Ciro Share 18', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table5.show_degrees_of_freedom(False)
print(table5.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Ciro Share 18 & -1.143$^{***}$ & 0.035$^{}$ & -0.029$^{**}$ & 0.009$^{}$ & -5.168$^{}$ & -0.384$^{}$ \\
  & (0.088) & (0.063) & (0.013) & (0.014) & (4.363) & (5.248) \\
 elderly(>70yo) & 0.057$^{***}$ & 0.065$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.361$^{***}$ & 2.787$^{***}$ \\
  & (0.003) & (0.005) & (0.001) & (0.001) & (0.265) & (0.362) \\
 Rural pop. & -0.001$^{***}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & -0.018$^{}$ & -0.013$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & 0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.020$^{***}$ & -0.020$^{***}$ & -0.001$^{}$ & -0

In [367]:
# Doses aplicadas ~ Tebet '22

# Create the OLS model with a different name
model1 = sm.OLS(y1, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.474
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     345.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                -1194.6
No. Observations:                5507   AIC:                             2425.
Df Residuals:                    5489   BIC:                             2544.
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -1.1058      0.683     -1.618      0.106      -2.445       0.234
Tebet Share 22            2.7306      0.320      8.530      0.000       2.103       3.358
elderly(>70yo)            0.0485      0.004     13.599      0.000       0.042       0.055
Rural pop.               -0.0002      0.000     -0.879      0.379      -0.001       0.000
White                    -0.0227      0.005     -5.031      0.000      -0.032      -0.014
Black                    -0.0238      0.005     -4.941      0.000      -0.033      -0.014
Brown                    -0.0272      0.005     -5.945      0.000      -0.036      -0.018
Indigenous               -0.0255      0.005     -5.528      0.000      -0.035      -0.016
Pop. density          -7.046e-06   1.09e-05     -0.647      0.517   -2.84e-05    1.43e-05
Catholic                  1.0329      0.071     14.577      0.000       0.894       1.172
Evangelic                 0.5924      0.110      5.388      0.000       0.377       0.808
Pentecostal Evangelic     0.9085      0.103      8.861      0.000       0.708       1.109
No religion               1.1673      0.142      8.210      0.000       0.889       1.446
Education 8y less         0.0362      0.005      6.626      0.000       0.025       0.047
Education 8-10y           0.0401      0.006      6.947      0.000       0.029       0.051
Education 11-14y          0.0454      0.006      8.136      0.000       0.034       0.056
Education 15y plus        0.0092      0.007      1.332      0.183      -0.004       0.023
Income (BRL)              0.0001   1.68e-05      7.158      0.000    8.74e-05       0.000
==============================================================================
Omnibus:                     3715.185   Durbin-Watson:                   1.466
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           591829.797
Skew:                           2.259   Prob(JB):                         0.00
Kurtosis:                      53.585   Cond. No.                     2.07e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [368]:
# Doses recebidas ~ Tebet '22

# Create the OLS model with a different name
model2 = sm.OLS(y2, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.573
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     333.7
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                -306.78
No. Observations:                5507   AIC:                             649.6
Df Residuals:                    5489   BIC:                             768.6
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.6523      0.497      5.339      0.000       1.679       3.626
Tebet Share 22            1.6337      0.294      5.557      0.000       1.058       2.210
elderly(>70yo)            0.0660      0.004     15.141      0.000       0.057       0.075
Rural pop.               -0.0014      0.000     -5.118      0.000      -0.002      -0.001
White                    -0.0203      0.004     -5.028      0.000      -0.028      -0.012
Black                    -0.0250      0.004     -5.803      0.000      -0.033      -0.017
Brown                    -0.0240      0.004     -5.895      0.000      -0.032      -0.016
Indigenous               -0.0228      0.004     -5.329      0.000      -0.031      -0.014
Pop. density          -4.697e-06   6.66e-06     -0.706      0.480   -1.77e-05    8.35e-06
Catholic                  1.2072      0.120     10.047      0.000       0.972       1.443
Evangelic                 0.6407      0.148      4.339      0.000       0.351       0.930
Pentecostal Evangelic     0.9964      0.177      5.618      0.000       0.649       1.344
No religion               1.1186      0.168      6.655      0.000       0.789       1.448
Education 8y less        -0.0054      0.003     -1.570      0.116      -0.012       0.001
Education 8-10y           0.0036      0.004      0.897      0.369      -0.004       0.011
Education 11-14y          0.0021      0.004      0.530      0.596      -0.006       0.010
Education 15y plus       -0.0315      0.005     -5.865      0.000      -0.042      -0.021
Income (BRL)              0.0001   1.64e-05      6.104      0.000    6.79e-05       0.000
==============================================================================
Omnibus:                      762.719   Durbin-Watson:                   1.841
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3745.950
Skew:                           0.579   Prob(JB):                         0.00
Kurtosis:                       6.871   Cond. No.                     2.07e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [369]:
# 3ª dose aplicada ~ Tebet '22

# Create the OLS model with a different name
model3 = sm.OLS(y3, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     260.8
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                 7550.9
No. Observations:                5507   AIC:                        -1.507e+04
Df Residuals:                    5489   BIC:                        -1.495e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.2947      0.119     -2.472      0.013      -0.528      -0.061
Tebet Share 22            0.4570      0.075      6.114      0.000       0.310       0.603
elderly(>70yo)            0.0179      0.001     22.236      0.000       0.016       0.019
Rural pop.                0.0002   6.19e-05      3.038      0.002    6.67e-05       0.000
White                    -0.0009      0.001     -0.927      0.354      -0.003       0.001
Black                    -0.0021      0.001     -2.042      0.041      -0.004   -8.54e-05
Brown                    -0.0017      0.001     -1.699      0.089      -0.004       0.000
Indigenous               -0.0008      0.001     -0.838      0.402      -0.003       0.001
Pop. density          -1.333e-06   1.16e-06     -1.149      0.250   -3.61e-06     9.4e-07
Catholic                  0.0904      0.014      6.429      0.000       0.063       0.118
Evangelic                 0.0033      0.028      0.118      0.906      -0.051       0.058
Pentecostal Evangelic     0.0976      0.028      3.476      0.001       0.043       0.153
No religion               0.1111      0.027      4.177      0.000       0.059       0.163
Education 8y less         0.0032      0.001      3.672      0.000       0.002       0.005
Education 8-10y           0.0043      0.001      4.059      0.000       0.002       0.006
Education 11-14y          0.0048      0.001      5.181      0.000       0.003       0.007
Education 15y plus       -0.0015      0.001     -1.064      0.287      -0.004       0.001
Income (BRL)           1.869e-05   4.46e-06      4.191      0.000    9.95e-06    2.74e-05
==============================================================================
Omnibus:                     2142.635   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            25531.656
Skew:                           1.519   Prob(JB):                         0.00
Kurtosis:                      13.101   Cond. No.                     2.07e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [370]:
# 3ª dose recebida ~ Tebet '22

# Create the OLS model with a different name
model4 = sm.OLS(y4, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     303.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:17   Log-Likelihood:                 8022.1
No. Observations:                5507   AIC:                        -1.601e+04
Df Residuals:                    5489   BIC:                        -1.589e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.1501      0.104     -1.438      0.150      -0.355       0.054
Tebet Share 22            0.4367      0.069      6.287      0.000       0.301       0.573
elderly(>70yo)            0.0180      0.001     23.625      0.000       0.016       0.019
Rural pop.             8.587e-05   5.53e-05      1.554      0.120   -2.24e-05       0.000
White                    -0.0010      0.001     -1.082      0.279      -0.003       0.001
Black                    -0.0023      0.001     -2.411      0.016      -0.004      -0.000
Brown                    -0.0018      0.001     -1.951      0.051      -0.004    8.45e-06
Indigenous               -0.0008      0.001     -0.928      0.354      -0.003       0.001
Pop. density          -9.314e-07   1.06e-06     -0.880      0.379   -3.01e-06    1.14e-06
Catholic                  0.1029      0.015      6.813      0.000       0.073       0.132
Evangelic                 0.0127      0.027      0.472      0.637      -0.040       0.065
Pentecostal Evangelic     0.1035      0.029      3.539      0.000       0.046       0.161
No religion               0.1122      0.025      4.417      0.000       0.062       0.162
Education 8y less         0.0018      0.001      2.493      0.013       0.000       0.003
Education 8-10y           0.0029      0.001      3.478      0.001       0.001       0.005
Education 11-14y          0.0035      0.001      4.297      0.000       0.002       0.005
Education 15y plus       -0.0031      0.001     -2.515      0.012      -0.005      -0.001
Income (BRL)           1.808e-05   3.94e-06      4.588      0.000    1.04e-05    2.58e-05
==============================================================================
Omnibus:                     1544.665   Durbin-Watson:                   1.807
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10454.168
Skew:                           1.166   Prob(JB):                         0.00
Kurtosis:                       9.334   Cond. No.                     2.07e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [371]:
# Cobertura vacinal ~ Tebet '22

# Create the OLS model with a different name
model5 = sm.OLS(y5, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     63.39
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.32e-198
Time:                        13:55:17   Log-Likelihood:                -24785.
No. Observations:                5507   AIC:                         4.961e+04
Df Residuals:                    5489   BIC:                         4.973e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   140.3198     41.653      3.369      0.001      58.682     221.958
Tebet Share 22          -27.2881     23.787     -1.147      0.251     -73.910      19.334
elderly(>70yo)            2.3101      0.263      8.798      0.000       1.795       2.825
Rural pop.               -0.0117      0.021     -0.559      0.576      -0.053       0.029
White                    -1.2478      0.335     -3.728      0.000      -1.904      -0.592
Black                    -1.4776      0.354     -4.174      0.000      -2.171      -0.784
Brown                    -1.4304      0.337     -4.243      0.000      -2.091      -0.770
Indigenous               -1.5686      0.340     -4.619      0.000      -2.234      -0.903
Pop. density             -0.0018      0.001     -3.001      0.003      -0.003      -0.001
Catholic                  9.3300      3.118      2.992      0.003       3.219      15.441
Evangelic                -1.4350      7.957     -0.180      0.857     -17.031      14.161
Pentecostal Evangelic    16.3431      6.260      2.611      0.009       4.074      28.612
No religion             -54.9140      9.616     -5.710      0.000     -73.762     -36.066
Education 8y less         0.4438      0.284      1.563      0.118      -0.113       1.001
Education 8-10y           0.5520      0.322      1.716      0.086      -0.079       1.183
Education 11-14y         -0.1058      0.316     -0.335      0.738      -0.725       0.514
Education 15y plus       -2.7886      0.483     -5.774      0.000      -3.735      -1.842
Income (BRL)              0.0129      0.001      8.802      0.000       0.010       0.016
==============================================================================
Omnibus:                      523.979   Durbin-Watson:                   1.878
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1975.315
Skew:                           0.429   Prob(JB):                         0.00
Kurtosis:                       5.806   Cond. No.                     2.07e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [372]:
# Cobertura contra Polio ~ Tebet '22

# Create the OLS model with a different name
model6 = sm.OLS(y6, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     36.24
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          2.56e-113
Time:                        13:55:18   Log-Likelihood:                -26172.
No. Observations:                5507   AIC:                         5.238e+04
Df Residuals:                    5489   BIC:                         5.250e+04
Df Model:                          17                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   180.7906     53.556      3.376      0.001      75.822     285.759
Tebet Share 22          -24.0849     30.837     -0.781      0.435     -84.525      36.355
elderly(>70yo)            2.7767      0.357      7.781      0.000       2.077       3.476
Rural pop.               -0.0108      0.026     -0.415      0.678      -0.062       0.040
White                    -1.3224      0.434     -3.044      0.002      -2.174      -0.471
Black                    -1.6028      0.458     -3.501      0.000      -2.500      -0.705
Brown                    -1.5223      0.436     -3.490      0.000      -2.377      -0.667
Indigenous               -1.7800      0.437     -4.075      0.000      -2.636      -0.924
Pop. density             -0.0014      0.001     -1.892      0.058      -0.003    4.87e-05
Catholic                 10.1788      3.926      2.593      0.010       2.484      17.873
Evangelic               -10.7482     12.901     -0.833      0.405     -36.033      14.537
Pentecostal Evangelic    12.3998      8.303      1.493      0.135      -3.873      28.672
No religion             -43.5850     12.427     -3.507      0.000     -67.941     -19.229
Education 8y less         0.2565      0.365      0.703      0.482      -0.459       0.972
Education 8-10y           0.2727      0.415      0.657      0.511      -0.540       1.086
Education 11-14y         -0.3448      0.399     -0.865      0.387      -1.126       0.437
Education 15y plus       -2.8717      0.627     -4.579      0.000      -4.101      -1.643
Income (BRL)              0.0104      0.002      4.971      0.000       0.006       0.014
==============================================================================
Omnibus:                     2141.727   Durbin-Watson:                   1.934
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34315.501
Skew:                           1.427   Prob(JB):                         0.00
Kurtosis:                      14.891   Cond. No.                     2.07e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.07e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [373]:
table6 = Stargazer([model1, model2, model3, model4, model5, model6])
table6.covariate_order(['Tebet Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table6.show_degrees_of_freedom(False)
print(table6.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Tebet Share 22 & 2.731$^{***}$ & 1.634$^{***}$ & 0.457$^{***}$ & 0.437$^{***}$ & -27.288$^{}$ & -24.085$^{}$ \\
  & (0.320) & (0.294) & (0.075) & (0.069) & (23.787) & (30.837) \\
 elderly(>70yo) & 0.048$^{***}$ & 0.066$^{***}$ & 0.018$^{***}$ & 0.018$^{***}$ & 2.310$^{***}$ & 2.777$^{***}$ \\
  & (0.004) & (0.004) & (0.001) & (0.001) & (0.263) & (0.357) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{}$ & -0.012$^{}$ & -0.011$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.023$^{***}$ & -0.020$^{***}$ & -0.001$^{}

In [374]:
# Doses aplicadas ~ Bolsonaro '18

# Create the OLS model with a different name
model1 = sm.OLS(y1, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     413.6
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:18   Log-Likelihood:                -1300.0
No. Observations:                5507   AIC:                             2630.
Df Residuals:                    5492   BIC:                             2729.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.8861      0.459      4.107      0.000       0.986       2.786
Bolsonaro Share 18        0.2539      0.046      5.493      0.000       0.163       0.345
elderly(>70yo)            0.0442      0.003     13.039      0.000       0.038       0.051
Rural pop.             8.816e-05      0.000      0.311      0.756      -0.000       0.001
White                    -0.0169      0.005     -3.761      0.000      -0.026      -0.008
Black                    -0.0162      0.005     -3.361      0.001      -0.026      -0.007
Brown                    -0.0217      0.005     -4.767      0.000      -0.031      -0.013
Indigenous               -0.0197      0.005     -4.293      0.000      -0.029      -0.011
Pop. density          -4.847e-06    1.1e-05     -0.439      0.661   -2.65e-05    1.68e-05
Catholic                  1.0671      0.072     14.896      0.000       0.927       1.207
Evangelic                 0.6655      0.104      6.421      0.000       0.462       0.869
Pentecostal Evangelic     1.0485      0.103     10.132      0.000       0.846       1.251
No religion               1.1080      0.137      8.085      0.000       0.839       1.377
Education 11-14y          0.0099      0.002      5.715      0.000       0.006       0.013
Income (BRL)           9.087e-05   1.58e-05      5.769      0.000       6e-05       0.000
==============================================================================
Omnibus:                     3365.538   Durbin-Watson:                   1.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           510094.336
Skew:                           1.918   Prob(JB):                         0.00
Kurtosis:                      49.993   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [375]:
# Doses recebidas ~ Bolsonaro '18

# Create the OLS model with a different name
model2 = sm.OLS(y2, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     393.3
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:18   Log-Likelihood:                -364.42
No. Observations:                5507   AIC:                             758.8
Df Residuals:                    5492   BIC:                             858.0
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.0970      0.419      5.008      0.000       1.276       2.918
Bolsonaro Share 18        0.1288      0.038      3.392      0.001       0.054       0.203
elderly(>70yo)            0.0605      0.004     14.534      0.000       0.052       0.069
Rural pop.               -0.0013      0.000     -4.754      0.000      -0.002      -0.001
White                    -0.0190      0.004     -4.733      0.000      -0.027      -0.011
Black                    -0.0233      0.004     -5.455      0.000      -0.032      -0.015
Brown                    -0.0232      0.004     -5.719      0.000      -0.031      -0.015
Indigenous               -0.0221      0.004     -5.232      0.000      -0.030      -0.014
Pop. density          -1.846e-06   7.49e-06     -0.246      0.805   -1.65e-05    1.28e-05
Catholic                  1.2328      0.119     10.377      0.000       1.000       1.466
Evangelic                 0.7097      0.142      5.009      0.000       0.432       0.987
Pentecostal Evangelic     1.0398      0.181      5.743      0.000       0.685       1.395
No religion               1.0458      0.166      6.316      0.000       0.721       1.370
Education 11-14y          0.0086      0.002      5.092      0.000       0.005       0.012
Income (BRL)           7.381e-05   1.65e-05      4.472      0.000    4.15e-05       0.000
==============================================================================
Omnibus:                      736.683   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3440.236
Skew:                           0.570   Prob(JB):                         0.00
Kurtosis:                       6.700   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [376]:
# 3ª dose aplicada ~ Bolsonaro '18

# Create the OLS model with a different name
model3 = sm.OLS(y3, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     302.4
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:18   Log-Likelihood:                 7490.9
No. Observations:                5507   AIC:                        -1.495e+04
Df Residuals:                    5492   BIC:                        -1.485e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0214      0.098     -0.219      0.827      -0.213       0.170
Bolsonaro Share 18        0.0109      0.009      1.169      0.242      -0.007       0.029
elderly(>70yo)            0.0169      0.001     21.825      0.000       0.015       0.018
Rural pop.                0.0002   6.17e-05      3.636      0.000       0.000       0.000
White                    -0.0002      0.001     -0.226      0.821      -0.002       0.002
Black                    -0.0013      0.001     -1.311      0.190      -0.003       0.001
Brown                    -0.0012      0.001     -1.202      0.230      -0.003       0.001
Indigenous               -0.0003      0.001     -0.307      0.759      -0.002       0.002
Pop. density          -1.582e-06   1.29e-06     -1.224      0.221   -4.12e-06    9.51e-07
Catholic                  0.0914      0.014      6.421      0.000       0.063       0.119
Evangelic                 0.0183      0.027      0.678      0.498      -0.035       0.071
Pentecostal Evangelic     0.1334      0.029      4.522      0.000       0.076       0.191
No religion               0.0967      0.026      3.701      0.000       0.045       0.148
Education 11-14y          0.0018      0.000      4.313      0.000       0.001       0.003
Income (BRL)           1.811e-05   4.41e-06      4.111      0.000    9.48e-06    2.67e-05
==============================================================================
Omnibus:                     2077.034   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22370.186
Skew:                           1.492   Prob(JB):                         0.00
Kurtosis:                      12.412   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [377]:
# 3ª dose recebida ~ Bolsonaro '18

# Create the OLS model with a different name
model4 = sm.OLS(y4, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     348.0
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:55:18   Log-Likelihood:                 7963.3
No. Observations:                5507   AIC:                        -1.590e+04
Df Residuals:                    5492   BIC:                        -1.580e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0059      0.090     -0.065      0.948      -0.182       0.170
Bolsonaro Share 18        0.0137      0.008      1.757      0.079      -0.002       0.029
elderly(>70yo)            0.0170      0.001     23.213      0.000       0.016       0.018
Rural pop.                0.0001   5.53e-05      2.180      0.029    1.22e-05       0.000
White                    -0.0004      0.001     -0.474      0.635      -0.002       0.001
Black                    -0.0017      0.001     -1.772      0.076      -0.003       0.000
Brown                    -0.0014      0.001     -1.536      0.125      -0.003       0.000
Indigenous               -0.0005      0.001     -0.506      0.613      -0.002       0.001
Pop. density          -9.758e-07   1.25e-06     -0.781      0.435   -3.42e-06    1.47e-06
Catholic                  0.1048      0.015      6.958      0.000       0.075       0.134
Evangelic                 0.0268      0.026      1.032      0.302      -0.024       0.078
Pentecostal Evangelic     0.1313      0.030      4.306      0.000       0.072       0.191
No religion               0.0984      0.025      3.943      0.000       0.050       0.147
Education 11-14y          0.0019      0.000      5.076      0.000       0.001       0.003
Income (BRL)           1.635e-05   3.89e-06      4.207      0.000    8.73e-06     2.4e-05
==============================================================================
Omnibus:                     1545.370   Durbin-Watson:                   1.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10046.981
Skew:                           1.179   Prob(JB):                         0.00
Kurtosis:                       9.183   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [378]:
# Cobertura vacinal ~ Bolsonaro '18

# Create the OLS model with a different name
model5 = sm.OLS(y5, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     74.38
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          8.79e-195
Time:                        13:55:18   Log-Likelihood:                -24808.
No. Observations:                5507   AIC:                         4.965e+04
Df Residuals:                    5492   BIC:                         4.975e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   159.8916     33.572      4.763      0.000      94.091     225.692
Bolsonaro Share 18       20.5304      3.114      6.592      0.000      14.427      26.634
elderly(>70yo)            2.0126      0.258      7.811      0.000       1.508       2.518
Rural pop.                0.0064      0.021      0.305      0.761      -0.035       0.047
White                    -1.0591      0.336     -3.154      0.002      -1.717      -0.401
Black                    -1.1731      0.354     -3.310      0.001      -1.868      -0.478
Brown                    -1.1762      0.339     -3.473      0.001      -1.840      -0.512
Indigenous               -1.3370      0.341     -3.921      0.000      -2.005      -0.669
Pop. density             -0.0018      0.001     -2.859      0.004      -0.003      -0.001
Catholic                 14.0830      3.259      4.321      0.000       7.695      20.471
Evangelic                 4.0686      7.908      0.515      0.607     -11.430      19.568
Pentecostal Evangelic     9.4593      6.386      1.481      0.139      -3.058      21.976
No religion             -56.8700      9.728     -5.846      0.000     -75.936     -37.804
Education 11-14y         -0.9843      0.135     -7.311      0.000      -1.248      -0.720
Income (BRL)              0.0057      0.001      4.047      0.000       0.003       0.008
==============================================================================
Omnibus:                      568.792   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2215.728
Skew:                           0.465   Prob(JB):                         0.00
Kurtosis:                       5.965   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [379]:
# Cobertura contra Polio ~ Bolsonaro '18

# Create the OLS model with a different name
model6 = sm.OLS(y6, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     44.31
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.56e-116
Time:                        13:55:18   Log-Likelihood:                -26184.
No. Observations:                5507   AIC:                         5.240e+04
Df Residuals:                    5492   BIC:                         5.250e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   183.7717     43.328      4.241      0.000      98.850     268.694
Bolsonaro Share 18       19.6319      4.134      4.749      0.000      11.530      27.734
elderly(>70yo)            2.5107      0.346      7.266      0.000       1.833       3.188
Rural pop.                0.0087      0.026      0.334      0.738      -0.042       0.060
White                    -1.1570      0.435     -2.659      0.008      -2.010      -0.304
Black                    -1.3284      0.457     -2.905      0.004      -2.225      -0.432
Brown                    -1.2889      0.438     -2.945      0.003      -2.147      -0.431
Indigenous               -1.5719      0.438     -3.588      0.000      -2.431      -0.713
Pop. density             -0.0014      0.001     -1.836      0.066      -0.003    9.26e-05
Catholic                 14.8133      4.095      3.618      0.000       6.788      22.838
Evangelic                -5.9030     12.951     -0.456      0.649     -31.287      19.481
Pentecostal Evangelic     4.6262      8.412      0.550      0.582     -11.861      21.113
No religion             -44.8650     12.515     -3.585      0.000     -69.393     -20.337
Education 11-14y         -1.0658      0.173     -6.168      0.000      -1.405      -0.727
Income (BRL)              0.0033      0.002      1.776      0.076      -0.000       0.007
==============================================================================
Omnibus:                     2177.904   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34729.974
Skew:                           1.461   Prob(JB):                         0.00
Kurtosis:                      14.951   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [380]:
table7 = Stargazer([model1, model2, model3, model4, model5, model6])
table7.covariate_order(['Bolsonaro Share 18', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table7.show_degrees_of_freedom(False)
print(table7.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Bolsonaro Share 18 & 0.254$^{***}$ & 0.129$^{***}$ & 0.011$^{}$ & 0.014$^{*}$ & 20.530$^{***}$ & 19.632$^{***}$ \\
  & (0.046) & (0.038) & (0.009) & (0.008) & (3.114) & (4.134) \\
 elderly(>70yo) & 0.044$^{***}$ & 0.061$^{***}$ & 0.017$^{***}$ & 0.017$^{***}$ & 2.013$^{***}$ & 2.511$^{***}$ \\
  & (0.003) & (0.004) & (0.001) & (0.001) & (0.258) & (0.346) \\
 Rural pop. & 0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & 0.006$^{}$ & 0.009$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.017$^{***}$ & -0.019$^{***}$ & -0.000$^{}

------------------------
Descriptive statistics

In [381]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Ciro Share 18', 'Far-left 22',
       'Far-right 22', 'PL coligation 22', 'Tebet Share 22',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage',
       'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous',
       'Pop. density', 'Catholic', 'Evangelic', 'Pentecostal Evangelic',
       'Umbanda/Candomble', 'No religion', 'Education 8-10y',
       'Education 11-14y', 'Education 15y plus', 'Income (BRL)',
       'Education 8y less', 'Outer wing 18', 'outer wing 22'],
      dtype='object')

In [382]:
print(df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &        max \\
\midrule
Bolsonaro Share 22    &  5507.0 &     0.439 &    0.187 &    0.061 &      0.890 \\
Far-left 18           &  5507.0 &     0.004 &    0.002 &    0.000 &      0.025 \\
Far-right 22          &  5507.0 &     0.004 &    0.004 &    0.000 &      0.032 \\
Dose1 Applied         &  5507.0 &     0.754 &    0.176 &    0.009 &      3.998 \\
Dose2 Applied         &  5507.0 &     0.620 &    0.182 &    0.000 &      3.257 \\
Dose3 Applied         &  5507.0 &     0.139 &    0.083 &    0.000 &      0.827 \\
Dose1 Received        &  5507.0 &     0.793 &    0.159 &    0.179 &      3.120 \\
Dose2 Received        &  5507.0 &     0.648 &    0.172 &    0.085 &      2.811 \\
Dose3 Received        &  5507.0 &     0.140 &    0.080 &    0.004 &      0.744 \\
Vaccination coverage  &  5507.0 &    76.939 &   24.189 &    2.280 &    245.580 \\
BCG coverage          &  5507.0 &    73.169 &   36.321 &    0.000 &    772.

<ipython-input-382-c504c304be65>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [383]:
df['Region'].unique()

array(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'],
      dtype=object)

In [384]:
filtered_df = df[df['Region'] == 'Norte']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  449.0 &     0.454 &    0.166 &    0.086 &     0.829 \\
Far-left 18           &  449.0 &     0.005 &    0.003 &    0.000 &     0.020 \\
Far-right 22          &  449.0 &     0.001 &    0.001 &    0.000 &     0.006 \\
Dose1 Applied         &  449.0 &     0.617 &    0.164 &    0.175 &     2.252 \\
Dose2 Applied         &  449.0 &     0.446 &    0.145 &    0.051 &     1.197 \\
Dose3 Applied         &  449.0 &     0.068 &    0.047 &    0.001 &     0.406 \\
Dose1 Received        &  449.0 &     0.617 &    0.173 &    0.179 &     1.636 \\
Dose2 Received        &  449.0 &     0.445 &    0.151 &    0.085 &     0.961 \\
Dose3 Received        &  449.0 &     0.066 &    0.043 &    0.004 &     0.309 \\
Vaccination coverage  &  449.0 &    68.026 &   22.440 &   19.030 &   136.230 \\
BCG coverage          &  449.0 &    75.867 &   31.241 &    5.130 &   235.470 \\
Polio coverage   

<ipython-input-384-ff39d407ecb8>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [385]:
filtered_df = df[df['Region'] == 'Nordeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &     mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  1787.0 &    0.235 &    0.089 &    0.061 &     0.623 \\
Far-left 18           &  1787.0 &    0.003 &    0.002 &    0.000 &     0.022 \\
Far-right 22          &  1787.0 &    0.002 &    0.001 &    0.000 &     0.008 \\
Dose1 Applied         &  1787.0 &    0.676 &    0.227 &    0.009 &     3.998 \\
Dose2 Applied         &  1787.0 &    0.530 &    0.205 &    0.000 &     3.257 \\
Dose3 Applied         &  1787.0 &    0.101 &    0.066 &    0.000 &     0.827 \\
Dose1 Received        &  1787.0 &    0.750 &    0.148 &    0.269 &     3.120 \\
Dose2 Received        &  1787.0 &    0.581 &    0.157 &    0.113 &     2.811 \\
Dose3 Received        &  1787.0 &    0.102 &    0.060 &    0.007 &     0.690 \\
Vaccination coverage  &  1787.0 &   68.263 &   21.967 &    5.100 &   214.260 \\
BCG coverage          &  1787.0 &   64.670 &   39.712 &    0.000 &   772.860 \\
Polio coverage   

<ipython-input-385-1965c469d4bc>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [386]:
filtered_df = df[df['Region'] == 'Sudeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &        max \\
\midrule
Bolsonaro Share 22    &  1666.0 &     0.511 &    0.129 &    0.155 &      0.812 \\
Far-left 18           &  1666.0 &     0.004 &    0.003 &    0.000 &      0.025 \\
Far-right 22          &  1666.0 &     0.007 &    0.004 &    0.000 &      0.023 \\
Dose1 Applied         &  1666.0 &     0.806 &    0.093 &    0.420 &      1.609 \\
Dose2 Applied         &  1666.0 &     0.681 &    0.123 &    0.172 &      1.395 \\
Dose3 Applied         &  1666.0 &     0.165 &    0.077 &    0.001 &      0.741 \\
Dose1 Received        &  1666.0 &     0.832 &    0.125 &    0.437 &      1.658 \\
Dose2 Received        &  1666.0 &     0.700 &    0.140 &    0.229 &      1.498 \\
Dose3 Received        &  1666.0 &     0.166 &    0.075 &    0.010 &      0.744 \\
Vaccination coverage  &  1666.0 &    80.422 &   23.858 &    2.280 &    210.880 \\
BCG coverage          &  1666.0 &    73.449 &   32.653 &    0.000 &    230.

<ipython-input-386-77f52f9ad7b6>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [387]:
filtered_df = df[df['Region'] == 'Sul']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  1159.0 &     0.598 &    0.115 &    0.282 &     0.890 \\
Far-left 18           &  1159.0 &     0.004 &    0.002 &    0.000 &     0.024 \\
Far-right 22          &  1159.0 &     0.005 &    0.003 &    0.000 &     0.032 \\
Dose1 Applied         &  1159.0 &     0.842 &    0.099 &    0.396 &     2.252 \\
Dose2 Applied         &  1159.0 &     0.734 &    0.108 &    0.272 &     1.950 \\
Dose3 Applied         &  1159.0 &     0.188 &    0.083 &    0.006 &     0.503 \\
Dose1 Received        &  1159.0 &     0.864 &    0.129 &    0.377 &     2.575 \\
Dose2 Received        &  1159.0 &     0.751 &    0.131 &    0.296 &     2.177 \\
Dose3 Received        &  1159.0 &     0.189 &    0.082 &    0.022 &     0.509 \\
Vaccination coverage  &  1159.0 &    87.193 &   23.442 &    5.760 &   245.580 \\
BCG coverage          &  1159.0 &    83.753 &   36.360 &    0.000 &   300.000 \\
Polio

<ipython-input-387-f4e8c2d7bdf7>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [388]:
filtered_df = df[df['Region'] == 'Centro-Oeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  446.0 &     0.564 &    0.101 &    0.174 &     0.811 \\
Far-left 18           &  446.0 &     0.004 &    0.002 &    0.000 &     0.014 \\
Far-right 22          &  446.0 &     0.003 &    0.001 &    0.000 &     0.008 \\
Dose1 Applied         &  446.0 &     0.785 &    0.134 &    0.370 &     2.081 \\
Dose2 Applied         &  446.0 &     0.635 &    0.142 &    0.304 &     1.730 \\
Dose3 Applied         &  446.0 &     0.135 &    0.076 &    0.000 &     0.447 \\
Dose1 Received        &  446.0 &     0.809 &    0.184 &    0.378 &     2.484 \\
Dose2 Received        &  446.0 &     0.655 &    0.171 &    0.297 &     1.990 \\
Dose3 Received        &  446.0 &     0.136 &    0.072 &    0.015 &     0.461 \\
Vaccination coverage  &  446.0 &    81.013 &   22.759 &   21.360 &   196.690 \\
BCG coverage          &  446.0 &    75.959 &   31.371 &    4.000 &   220.000 \\
Polio coverage   

<ipython-input-388-a0e93a3389ef>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
